<a href="https://colab.research.google.com/github/Saxena224pawan/Alex_paper_testing_benchmark/blob/main/alex_vs_btree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Alex Code**

In [ ]:
%%writefile alex.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define NUM_OPERATIONS 1000000
#define LEAF_CAPACITY NUM_OPERATIONS
#define LINE_LEN 128

// ALEX Structures
typedef struct {
    float a, b;
} LinearModel;

typedef struct {
    int *keys;
    int *values;
    int size;
    int capacity;
    LinearModel model;
} ALEXLeaf;

void alex_init(ALEXLeaf *leaf, int initial_capacity) {
    //printf("Attempting to allocate ALEXLeaf with capacity: %d\n", initial_capacity);
    leaf->keys = (int *)malloc((size_t)initial_capacity * sizeof(int));
    leaf->values = (int *)malloc((size_t)initial_capacity * sizeof(int));
    if (!leaf->keys || !leaf->values) {
        perror("Failed to allocate memory for ALEXLeaf keys or values");
        free(leaf->keys);
        free(leaf->values);
        exit(EXIT_FAILURE);
    }
    leaf->size = 0;
    leaf->capacity = initial_capacity;
    leaf->model.a = 0.0;
    leaf->model.b = 0.0;
    //printf("ALEXLeaf initialized with capacity: %d\n", initial_capacity);
}

void alex_destroy(ALEXLeaf *leaf) {
    if (leaf) {
        //printf("Destroying ALEXLeaf data at address %p\n", (void*)leaf->keys);
        free(leaf->keys);
        free(leaf->values);
        leaf->keys = NULL;
        leaf->values = NULL;
        leaf->size = 0;
        leaf->capacity = 0;
    }
}

void train_model(LinearModel *model, int *keys, int n) {
    if (n == 0) { model->a = 0.0; model->b = 0.0; return; }
    if (n == 1) { model->a = 0.0; model->b = 0.0; return; } // Handle single element case gracefully
    int min_x = keys[0];
    int max_x = keys[n - 1];
    if (max_x == min_x) { model->a = 0.0; model->b = (n - 1) / 2.0; }
    else { model->a = (float)(n - 1) / (max_x - min_x); model->b = -model->a * min_x; }
}

int predict(LinearModel *model, int key) {
    return (int)(model->a * key + model->b);
}

void alex_insert(ALEXLeaf *leaf, int key, int value) {
    if (leaf->size >= leaf->capacity) {
        fprintf(stderr, "ALEXLeaf full (key %d). Current size: %d, Max capacity: %d. Further inserts will be ignored.\n", key, leaf->size, leaf->capacity);
        return;
    }

    int pos = predict(&leaf->model, key);
    if (pos < 0) pos = 0;
    if (pos > leaf->size) pos = leaf->size;
    while (pos < leaf->size && leaf->keys[pos] < key) pos++;
    for (int i = leaf->size; i > pos; i--) {
        leaf->keys[i] = leaf->keys[i - 1];
        leaf->values[i] = leaf->values[i - 1];
    }
    leaf->keys[pos] = key;
    leaf->values[pos] = value;
    leaf->size++;
    train_model(&leaf->model, leaf->keys, leaf->size);
}

int alex_lookup(ALEXLeaf *leaf, int key) {
    for (int i = 0; i < leaf->size; i++) {
        if (leaf->keys[i] == key) return leaf->values[i];
    }
    return -1;
}

void alex_remove(ALEXLeaf *leaf, int key) {
    for (int i = 0; i < leaf->size; i++) {
        if (leaf->keys[i] == key) {
            for (int j = i; j < leaf->size - 1; j++) {
                leaf->keys[j] = leaf->keys[j + 1];
                leaf->values[j] = leaf->values[j + 1];
            }
            leaf->size--;
            train_model(&leaf->model, leaf->keys, leaf->size);
            break;
        }
    }
}

int alex_lowerbound(ALEXLeaf *leaf, int key) {
    for (int i = 0; i < leaf->size; i++) {
        if (leaf->keys[i] >= key) return leaf->keys[i];
    }
    return -1;
}

// Dynamic List Helper Functions (duplicated for self-contained files)
typedef struct {
    int key;
    int value;
} InsertOp;

InsertOp *insert_ops_list = NULL;
long long insert_ops_count = 0;
long long insert_ops_capacity = 0;

int *lookup_keys_list = NULL;
long long lookup_keys_count = 0;
long long lookup_keys_capacity = 0;

int *lowerbound_keys_list = NULL;
long long lowerbound_keys_count = 0;
long long lowerbound_keys_capacity = 0;

int *remove_keys_list = NULL;
long long remove_keys_count = 0;
long long remove_keys_capacity = 0;

void resize_int_array(int **arr, long long *count, long long *capacity, long long min_needed_capacity) {
    if (*count >= *capacity) {
        long long new_cap = (*capacity == 0) ? NUM_OPERATIONS : *capacity * 2;
        if (new_cap < min_needed_capacity) {
             new_cap = min_needed_capacity;
        }
        int *new_arr = realloc(*arr, (size_t)new_cap * sizeof(int));
        if (!new_arr) {
            perror("Failed to reallocate integer array");
            exit(EXIT_FAILURE);
        }
        *arr = new_arr;
        *capacity = new_cap;
    }
}

void resize_insert_op_array(InsertOp **arr, long long *count, long long *capacity, long long min_needed_capacity) {
    if (*count >= *capacity) {
        long long new_cap = (*capacity == 0) ? NUM_OPERATIONS : *capacity * 2;
        if (new_cap < min_needed_capacity) {
             new_cap = min_needed_capacity;
        }
        InsertOp *new_arr = realloc(*arr, (size_t)new_cap * sizeof(InsertOp));
        if (!new_arr) {
            perror("Failed to reallocate InsertOp array");
            exit(EXIT_FAILURE);
        }
        *arr = new_arr;
        *capacity = new_cap;
    }
}

void add_insert_op(int key, int value) {
    resize_insert_op_array(&insert_ops_list, &insert_ops_count, &insert_ops_capacity, insert_ops_count + 1);
    insert_ops_list[insert_ops_count].key = key;
    insert_ops_list[insert_ops_count].value = value;
    insert_ops_count++;
}

void add_lookup_key(int key) {
    resize_int_array(&lookup_keys_list, &lookup_keys_count, &lookup_keys_capacity, lookup_keys_count + 1);
    lookup_keys_list[lookup_keys_count++] = key;
}

void add_lowerbound_key(int key) {
    resize_int_array(&lowerbound_keys_list, &lowerbound_keys_count, &lowerbound_keys_capacity, lowerbound_keys_count + 1);
    lowerbound_keys_list[lowerbound_keys_count++] = key;
}

void add_remove_key(int key) {
    resize_int_array(&remove_keys_list, &remove_keys_count, &remove_keys_capacity, remove_keys_count + 1);
    remove_keys_list[remove_keys_count++] = key;
}

void parse_and_collect_operations(const char *filename) {
    FILE *fp = fopen(filename, "r");
    if (!fp) {
        fprintf(stderr, "ERROR: Failed to open file %s. Please ensure it exists and is accessible.\n", filename);
        exit(EXIT_FAILURE);
    }

    char line[LINE_LEN];
    long long line_num = 0;
    printf("Parsing and collecting operations from file: %s for ALEX...\n", filename);

    while (fgets(line, sizeof(line), fp)) {
        line_num++;
        line[strcspn(line, "\r\n")] = 0;

        if (strncmp(line, "insert", 6) == 0) {
            int k, v;
            if (sscanf(line, "insert %d %d", &k, &v) == 2) {
                add_insert_op(k, v);
                if (!fgets(line, sizeof(line), fp)) break; // Consume 'ok' line
            } else {
                fprintf(stderr, "ERROR: Malformed insert line on line %lld: '%s'\n", line_num, line);
            }
        } else if (strncmp(line, "lookup", 6) == 0) {
            int k;
            if (sscanf(line, "lookup %d", &k) == 1) {
                add_lookup_key(k);
                if (!fgets(line, sizeof(line), fp)) break; // Consume result line
            } else {
                fprintf(stderr, "ERROR: Malformed lookup line on line %lld: '%s'\n", line_num, line);
            }
        } else if (strncmp(line, "remove", 6) == 0) {
            int k;
            if (sscanf(line, "remove %d", &k) == 1) {
                add_remove_key(k);
                if (!fgets(line, sizeof(line), fp)) break; // Consume 'ok' line
            } else {
                fprintf(stderr, "ERROR: Malformed remove line on line %lld: '%s'\n", line_num, line);
            }
        } else if (strncmp(line, "lowerbound", 10) == 0) {
            int k;
            if (sscanf(line, "lowerbound %d", &k) == 1) {
                add_lowerbound_key(k);
                if (!fgets(line, sizeof(line), fp)) break; // Consume result line
            } else {
                fprintf(stderr, "ERROR: Malformed lowerbound line on line %lld: '%s'\n", line_num, line);
            }
        } else if (strlen(line) > 0 && line[0] != '\n' && line[0] != '\r') {
            //fprintf(stderr, "WARNING: Unrecognized or invalid command on line %lld: '%s'\n", line_num, line);
        }
    }
    fclose(fp);
    printf("Finished parsing for ALEX. Collected %lld inserts, %lld lookups, %lld lowerbounds, %lld removals.\n",
           insert_ops_count, lookup_keys_count, lowerbound_keys_count, remove_keys_count);
}

double get_elapsed_time(struct timespec start, struct timespec end) {
    return (double)(end.tv_sec - start.tv_sec) + (double)(end.tv_nsec - start.tv_nsec) / 1e9;
}

int main(int argc, char *argv[]) {
    srand((unsigned int)time(NULL));

    if (argc != 2) {
        printf("Usage: %s <benchmark_operations_file>\n", argv[0]);
        return 1;
    }

    printf("--- Initializing ALEX Data Structure ---\n");
    ALEXLeaf *alex = (ALEXLeaf *)malloc(sizeof(ALEXLeaf));
    if (!alex) {
        perror("Failed to allocate ALEXLeaf structure");
        return 1;
    }
    alex_init(alex, LEAF_CAPACITY);

    parse_and_collect_operations(argv[1]);

    printf("\n--- Starting ALEX Benchmarks ---\n");
    struct timespec start_total, end_total;
    clock_gettime(CLOCK_MONOTONIC, &start_total);

    // Insert Benchmark
    printf("\nAttempting to run ALEX Insert Benchmark (%lld operations)...\n", insert_ops_count);
    struct timespec start_insert, end_insert;
    clock_gettime(CLOCK_MONOTONIC, &start_insert);
    for (long long i = 0; i < insert_ops_count; ++i) {
        alex_insert(alex, insert_ops_list[i].key, insert_ops_list[i].value);
    }
    clock_gettime(CLOCK_MONOTONIC, &end_insert);
    printf("ALEX Insert time: %.6f seconds\n", get_elapsed_time(start_insert, end_insert));
    printf("ALEX Insert Benchmark function call completed.\n");

    // Lookup Benchmark
    printf("\nAttempting to run ALEX Lookup Benchmark (%lld operations)...\n", lookup_keys_count);
    struct timespec start_lookup, end_lookup;
    clock_gettime(CLOCK_MONOTONIC, &start_lookup);
    for (long long i = 0; i < lookup_keys_count; ++i) {
        alex_lookup(alex, lookup_keys_list[i]);
    }
    clock_gettime(CLOCK_MONOTONIC, &end_lookup);
    printf("ALEX Lookup time: %.6f seconds\n", get_elapsed_time(start_lookup, end_lookup));
    printf("ALEX Lookup Benchmark function call completed.\n");

    // Lowerbound Benchmark
    printf("\nAttempting to run ALEX Lowerbound Benchmark (%lld operations)...\n", lowerbound_keys_count);
    struct timespec start_lowerbound, end_lowerbound;
    clock_gettime(CLOCK_MONOTONIC, &start_lowerbound);
    for (long long i = 0; i < lowerbound_keys_count; ++i) {
        alex_lowerbound(alex, lowerbound_keys_list[i]);
    }
    clock_gettime(CLOCK_MONOTONIC, &end_lowerbound);
    printf("ALEX Lowerbound time: %.6f seconds\n", get_elapsed_time(start_lowerbound, end_lowerbound));
    printf("ALEX Lowerbound Benchmark function call completed.\n");

    // Remove Benchmark
    printf("\nAttempting to run ALEX Remove Benchmark (%lld operations)...\n", remove_keys_count);
    struct timespec start_remove, end_remove;
    clock_gettime(CLOCK_MONOTONIC, &start_remove);
    for (long long i = 0; i < remove_keys_count; ++i) {
        alex_remove(alex, remove_keys_list[i]);
    }
    clock_gettime(CLOCK_MONOTONIC, &end_remove);
    printf("ALEX Remove time: %.6f seconds\n", get_elapsed_time(start_remove, end_remove));
    printf("ALEX Remove Benchmark function call completed.\n");

    clock_gettime(CLOCK_MONOTONIC, &end_total);
    printf("\nTotal ALEX Benchmark time: %.6f seconds\n", get_elapsed_time(start_total, end_total));

    printf("\n--- Starting ALEX Memory Cleanup ---\n");
    if (insert_ops_list) { free(insert_ops_list); insert_ops_list = NULL; }
    if (lookup_keys_list) { free(lookup_keys_list); lookup_keys_list = NULL; }
    if (lowerbound_keys_list) { free(lowerbound_keys_list); lowerbound_keys_list = NULL; }
    if (remove_keys_list) { free(remove_keys_list); remove_keys_list = NULL; }
    if (alex) {
        alex_destroy(alex);
        free(alex);
        alex = NULL;
    }
    printf("ALEX Memory cleanup completed.\n");
    printf("\nAll ALEX benchmarks and memory cleanup completed.\n");

    return 0;
}

Writing alex.c


In [ ]:
!gcc alex.c -o alex

In [ ]:
!./alex combined_benchmark_200k_ops.txt

--- Initializing ALEX Data Structure ---
Parsing and collecting operations from file: combined_benchmark_200k_ops.txt for ALEX...
Finished parsing for ALEX. Collected 100000 inserts, 100000 lookups, 100000 lowerbounds, 100000 removals.

--- Starting ALEX Benchmarks ---

Attempting to run ALEX Insert Benchmark (100000 operations)...
ALEX Insert time: 10.963480 seconds
ALEX Insert Benchmark function call completed.

Attempting to run ALEX Lookup Benchmark (100000 operations)...
ALEX Lookup time: 19.550919 seconds
ALEX Lookup Benchmark function call completed.

Attempting to run ALEX Lowerbound Benchmark (100000 operations)...
ALEX Lowerbound time: 11.497734 seconds
ALEX Lowerbound Benchmark function call completed.

Attempting to run ALEX Remove Benchmark (100000 operations)...
ALEX Remove time: 18.387723 seconds
ALEX Remove Benchmark function call completed.

Total ALEX Benchmark time: 60.400001 seconds

--- Starting ALEX Memory Cleanup ---
ALEX Memory cleanup completed.

All ALEX bench

In [ ]:
!./alex combined_benchmark_2m_ops.txt

--- Initializing ALEX Data Structure ---
Parsing and collecting operations from file: combined_benchmark_2m_ops.txt for ALEX...
Finished parsing for ALEX. Collected 1000000 inserts, 1000000 lookups, 1000000 lowerbounds, 1000000 removals.

--- Starting ALEX Benchmarks ---

Attempting to run ALEX Insert Benchmark (1000000 operations)...
ALEX Insert time: 1106.244725 seconds
ALEX Insert Benchmark function call completed.

Attempting to run ALEX Lookup Benchmark (1000000 operations)...
ALEX Lookup time: 2010.224513 seconds
ALEX Lookup Benchmark function call completed.

Attempting to run ALEX Lowerbound Benchmark (1000000 operations)...
ALEX Lowerbound time: 1149.832327 seconds
ALEX Lowerbound Benchmark function call completed.

Attempting to run ALEX Remove Benchmark (1000000 operations)...


## **BTree code**

In [ ]:
%%writefile btree.c
/*
 * btree_benchmark.c (B-tree Index Benchmark with Separate Operation Timings)
 * -------------------------------------------------------------------------
 * This benchmark reads all commands from a single input file, categorizes them,
 * and then runs separate benchmark phases for Insert, Lookup, Lowerbound, and Remove
 * operations, reporting individual execution times for the B-tree.
 *
 * This version is configured for 1,000,000 operations per benchmark phase.
 * Adapted for Linux/POSIX environments (e.g., Google Colab) using clock_gettime for timing.
 */

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h> // For time() to seed random number generator and clock_gettime

#define NUM_OPERATIONS 1000000
#define ORDER 32            // Order of the B+Tree (minimum number of keys in a non-root node is ORDER-1, max is 2*ORDER-1)
#define LINE_LEN 128        // Max length of a line when reading from file.

/* ===================== Global Operation Lists for Collection ===================== */

// Structure to hold an insert operation's key and value
typedef struct {
    int key;
    int value;
} InsertOp;

// Dynamic arrays to store all operations parsed from the input file
InsertOp *insert_ops_list = NULL;
long long insert_ops_count = 0;
long long insert_ops_capacity = 0;

int *lookup_keys_list = NULL;
long long lookup_keys_count = 0;
long long lookup_keys_capacity = 0;

int *lowerbound_keys_list = NULL;
long long lowerbound_keys_count = 0;
long long lowerbound_keys_capacity = 0;

int *remove_keys_list = NULL;
long long remove_keys_count = 0;
long long remove_keys_capacity = 0;

/* ===================== B+Tree Structures ===================== */

typedef struct BTreeNode {
    int keys[2 * ORDER - 1];
    int values[2 * ORDER - 1];
    struct BTreeNode *children[2 * ORDER];
    int leaf;
    int num_keys;
} BTreeNode;

BTreeNode* create_btree_node(int leaf) {
    BTreeNode *node = (BTreeNode *)malloc(sizeof(BTreeNode));
    if (!node) {
        perror("Failed to allocate BTreeNode");
        exit(EXIT_FAILURE);
    }
    node->leaf = leaf;
    node->num_keys = 0;
    for (int i = 0; i < 2 * ORDER; i++)
        node->children[i] = NULL;
    return node;
}

void btree_split_child(BTreeNode *parent, int i);
void btree_insert_nonfull(BTreeNode *node, int key, int value);

void btree_insert(BTreeNode **root, int key, int value) {
    BTreeNode *r = *root;
    if (r->num_keys == 2 * ORDER - 1) { // Root is full
        BTreeNode *s = create_btree_node(0); // Create new root
        s->children[0] = r;
        btree_split_child(s, 0); // Split old root and connect to new root
        *root = s; // Update root pointer
        btree_insert_nonfull(s, key, value); // Insert into new root's appropriate child
    } else {
        btree_insert_nonfull(r, key, value); // Insert into non-full root
    }
}

void btree_insert_nonfull(BTreeNode *node, int key, int value) {
    int i = node->num_keys - 1; // Start from the rightmost key

    if (node->leaf) {
        // If it's a leaf node, shift keys and values to make space
        while (i >= 0 && key < node->keys[i]) {
            node->keys[i + 1] = node->keys[i];
            node->values[i + 1] = node->values[i];
            i--;
        }
        node->keys[i + 1] = key;   // Insert key
        node->values[i + 1] = value; // Insert value
        node->num_keys++;          // Increment key count
    } else {
        // If it's an internal node, find the correct child
        while (i >= 0 && key < node->keys[i]) i--;
        i++; // i is now the index of the child pointer to descend into

        // If the child is full, split it
        if (node->children[i]->num_keys == 2 * ORDER - 1) {
            btree_split_child(node, i);
            // After splitting, the parent might have a new key at index 'i'.
            // Decide which of the two new children to descend into.
            if (key > node->keys[i]) i++;
        }
        // Recursively insert into the appropriate child
        btree_insert_nonfull(node->children[i], key, value);
    }
}

/**
 * @brief Splits a full child of a B-tree node.
 * This function assumes `parent->children[i]` (let's call it `y`) is full (2*ORDER - 1 keys).
 * It splits `y` into two nodes (`y` and `z`), moves the median key to `parent`,
 * and adjusts parent's children and key counts.
 *
 * @param parent Pointer to the parent node.
 * @param i Index of the child that needs to be split.
 */
void btree_split_child(BTreeNode *parent, int i) {
    BTreeNode *y = parent->children[i];         // The child to be split (full node)
    BTreeNode *z = create_btree_node(y->leaf); // Create a new node for the right half

    // Copy the upper half of keys (from median+1 to end) from y to z
    // In a B-tree with max 2*ORDER-1 keys, the median is at index ORDER-1.
    // Keys to move to z are from index ORDER to 2*ORDER-2 (inclusive), which are ORDER-1 keys.
    for (int j = 0; j < ORDER - 1; j++) {
        z->keys[j] = y->keys[j + ORDER];
        z->values[j] = y->values[j + ORDER];
    }

    // If y is an internal node, copy the upper half of its children pointers to z
    // Children to move are from index ORDER to 2*ORDER-1 (inclusive), which are ORDER children.
    if (!y->leaf) {
        for (int j = 0; j < ORDER; j++) {
            z->children[j] = y->children[j + ORDER];
        }
    }

    // Set the number of keys for the new node z
    z->num_keys = ORDER - 1; // The new node has ORDER-1 keys

    // Update the number of keys for the original node y
    // y retains keys from index 0 to ORDER-2 (inclusive), which are ORDER-1 keys.
    y->num_keys = ORDER - 1;

    // Shift children pointers in the parent to make space for the new node z
    for (int j = parent->num_keys; j >= i + 1; j--)
        parent->children[j + 1] = parent->children[j];
    parent->children[i + 1] = z; // Assign the new node z as a child of parent

    // Shift keys and values in the parent to make space for the median key
    // The median key (from y->keys[ORDER-1]) will be inserted into parent->keys[i]
    for (int j = parent->num_keys - 1; j >= i; j--) {
        parent->keys[j + 1] = parent->keys[j];
        parent->values[j + 1] = parent->values[j]; // Values are also handled if stored in internal nodes
    }
    // Move the median key and its value from y to the parent
    parent->keys[i] = y->keys[ORDER - 1];
    parent->values[i] = y->values[ORDER - 1];

    parent->num_keys++; // Parent gains one key
}

int btree_lookup(BTreeNode *node, int key) {
    int i = 0;
    while (i < node->num_keys && key > node->keys[i]) i++;
    if (i < node->num_keys && key == node->keys[i]) return node->values[i];
    if (node->leaf) return -1;
    return btree_lookup(node->children[i], key);
}

/**
 * @brief Finds the smallest key in the B+Tree that is greater than or equal to the given key.
 * This is a recursive function that traverses the tree.
 * @param node Pointer to the current node being searched.
 * @param key The key for which to find the lower bound.
 * @return The lower bound key, or -1 if no such key exists.
 */
int btree_lowerbound(BTreeNode *node, int key) {
    int i = 0;
    // Find the first key in the current node that is greater than or equal to the target key.
    // Or find the appropriate child pointer to descend into.
    while (i < node->num_keys && key > node->keys[i]) {
        i++;
    }

    // If 'i' is within the bounds of keys and the key at 'i' is >= target key
    if (i < node->num_keys && node->keys[i] >= key) {
        if (node->leaf) {
            // If it's a leaf node, this is the first key >= target, so it's our lower bound.
            return node->keys[i];
        } else {
            // If it's an internal node, the lower bound could be in the child pointed to by children[i]
            // or it could be the key at node->keys[i] itself.
            int result_in_child = btree_lowerbound(node->children[i], key);
            if (result_in_child != -1) {
                // If a lower bound was found in the child, that's our result.
                return result_in_child;
            } else {
                // If no lower bound found in the child, it means all keys in that child's subtree
                // are less than 'key'. So, the current key at node->keys[i] is the smallest
                // key in this subtree that is greater than or equal to 'key'.
                return node->keys[i];
            }
        }
    } else {
        // If 'key' is greater than all keys in this node, and we are an internal node,
        // we must check the last child (node->children[node->num_keys]) for the lower bound.
        if (!node->leaf) {
            return btree_lowerbound(node->children[node->num_keys], key);
        } else {
            // If it's a leaf node and 'key' is greater than all its keys,
            // then no key greater than or equal to 'key' exists in this leaf or its subtree.
            return -1;
        }
    }
}


// Forward declarations for B-tree removal functions
BTreeNode* btree_remove_recursive(BTreeNode *node, int key);
int btree_get_pred(BTreeNode *node, int idx);
int btree_get_succ(BTreeNode *node, int idx);
void btree_fill(BTreeNode *node, int idx);
void btree_borrow_from_prev(BTreeNode *node, int idx);
void btree_borrow_from_next(BTreeNode *node, int idx);
void btree_merge(BTreeNode *node, int idx);


/* ===================== B-Tree Deletion Helper Functions ===================== */

/**
 * @brief Finds the predecessor key for a key at a given index in an internal node.
 * The predecessor is the largest key in the left subtree of the key being deleted.
 * This is found by traversing to the rightmost leaf of the left child.
 * @param node Pointer to the parent node containing the key.
 * @param idx The index of the key in `node->keys` for which to find the predecessor.
 * @return The predecessor key.
 */
int btree_get_pred(BTreeNode *node, int idx) {
    // Start from the child node to the left of the key (node->children[idx])
    BTreeNode *curr = node->children[idx];
    // Traverse down to the rightmost leaf node in this subtree
    while (!curr->leaf) {
        curr = curr->children[curr->num_keys];
    }
    // The predecessor is the last key in this leaf node
    return curr->keys[curr->num_keys - 1];
}

/**
 * @brief Finds the successor key for a key at a given index in an internal node.
 * The successor is the smallest key in the right subtree of the key being deleted.
 * This is found by traversing to the leftmost leaf of the right child.
 * @param node Pointer to the parent node containing the key.
 * @param idx The index of the key in `node->keys` for which to find the successor.
 * @return The successor key.
 */
int btree_get_succ(BTreeNode *node, int idx) {
    // Start from the child node to the right of the key (node->children[idx + 1])
    BTreeNode *curr = node->children[idx + 1];
    // Traverse down to the leftmost leaf node in this subtree
    while (!curr->leaf) {
        curr = curr->children[0];
    }
    // The successor is the first key in this leaf node
    return curr->keys[0];
}

/**
 * @brief Borrows a key from the left sibling (previous sibling) of child 'idx'.
 * This operation is performed when `node->children[idx]` is deficient (has `ORDER-1` keys)
 * and its left sibling (`node->children[idx-1]`) has at least `ORDER` keys.
 * A key moves from the sibling to the parent, and a key from the parent moves to the deficient child.
 * @param node Pointer to the parent node.
 * @param idx Index of the child that needs a key.
 */
void btree_borrow_from_prev(BTreeNode *node, int idx) {
    BTreeNode *child = node->children[idx];        // The child that is deficient
    BTreeNode *sibling = node->children[idx - 1]; // The generous left sibling

    // 1. Shift all keys and values in the deficient child one position to the right
    //    to make space for the incoming key from the parent.
    for (int i = child->num_keys - 1; i >= 0; --i) {
        child->keys[i + 1] = child->keys[i];
        child->values[i + 1] = child->values[i];
    }

    // 2. If the deficient child is an internal node, shift its children pointers too.
    if (!child->leaf) {
        for (int i = child->num_keys; i >= 0; --i) {
            child->children[i + 1] = child->children[i];
        }
    }

    // 3. Move the separating key from the parent (node->keys[idx-1]) down to the
    //    deficient child's first position (index 0).
    child->keys[0] = node->keys[idx - 1];
    child->values[0] = node->values[idx - 1]; // If internal nodes store values, copy it too.

    // 4. If the deficient child is internal, move the rightmost child pointer
    //    from the sibling to the deficient child's first child pointer.
    if (!child->leaf) {
        child->children[0] = sibling->children[sibling->num_keys];
    }

    // 5. Move the rightmost key from the sibling (sibling->keys[sibling->num_keys - 1])
    //    up to the parent, replacing the key that moved down.
    node->keys[idx - 1] = sibling->keys[sibling->num_keys - 1];
    node->values[idx - 1] = sibling->values[sibling->num_keys - 1]; // Copy value too.

    child->num_keys++;    // Deficient child gains a key
    sibling->num_keys--;  // Sibling loses a key
}

/**
 * @brief Borrows a key from the right sibling (next sibling) of child 'idx'.
 * This operation is performed when `node->children[idx]` is deficient (has `ORDER-1` keys)
 * and its right sibling (`node->children[idx+1]`) has at least `ORDER` keys.
 * A key moves from the sibling to the parent, and a key from the parent moves to the deficient child.
 * @param node Pointer to the parent node.
 * @param idx Index of the child that needs a key.
 */
void btree_borrow_from_next(BTreeNode *node, int idx) {
    BTreeNode *child = node->children[idx];        // The child that is deficient
    BTreeNode *sibling = node->children[idx + 1]; // The generous right sibling

    // 1. Move the separating key from the parent (node->keys[idx]) down to the
    //    deficient child's last position.
    child->keys[child->num_keys] = node->keys[idx];
    child->values[child->num_keys] = node->values[idx]; // If internal nodes store values.

    // 2. If the deficient child is internal, move the leftmost child pointer
    //    from the sibling to the deficient child's last child pointer.
    if (!child->leaf) {
        child->children[child->num_keys + 1] = sibling->children[0];
    }

    // 3. Move the leftmost key from the sibling (sibling->keys[0])
    //    up to the parent, replacing the key that moved down.
    node->keys[idx] = sibling->keys[0];
    node->values[idx] = sibling->values[0]; // Copy value too.

    // 4. Shift all keys and values in the sibling one position to the left,
    //    to fill the gap created by moving its first key.
    for (int i = 1; i < sibling->num_keys; ++i) {
        sibling->keys[i - 1] = sibling->keys[i];
        sibling->values[i - 1] = sibling->values[i];
    }
    // 5. If the sibling is an internal node, shift its children pointers too.
    if (!sibling->leaf) {
        for (int i = 1; i <= sibling->num_keys; ++i) {
            sibling->children[i - 1] = sibling->children[i];
        }
    }

    child->num_keys++;    // Deficient child gains a key
    sibling->num_keys--;  // Sibling loses a key
}

/**
 * @brief Merges child node[idx] with its right sibling (child node[idx+1]).
 * This operation is performed when a child is deficient and neither sibling
 * can lend a key (both have `ORDER-1` keys).
 * The separating key from the parent (`node->keys[idx]`) is also pulled down
 * into the merged node. The right sibling is then freed.
 * @param node Pointer to the parent node.
 * @param idx Index of the left child involved in the merge.
 */
void btree_merge(BTreeNode *node, int idx) {
    BTreeNode *child = node->children[idx];        // The left child (will absorb the sibling)
    BTreeNode *sibling = node->children[idx + 1]; // The right sibling (will be merged into 'child')

    // 1. Pull the separating key from the parent (node->keys[idx]) down
    //    to the end of the left child's keys.
    child->keys[child->num_keys] = node->keys[idx];
    child->values[child->num_keys] = node->values[idx]; // If internal nodes store values.
    child->num_keys++; // Increment key count of the left child for the pulled-down key.

    // 2. Copy all keys and values from the right sibling to the left child.
    for (int i = 0; i < sibling->num_keys; ++i) {
        child->keys[child->num_keys + i] = sibling->keys[i];
        child->values[child->num_keys + i] = sibling->values[i]; // If internal nodes store values.
    }

    // 3. If the nodes are internal, copy all child pointers from the right sibling to the left child.
    if (!child->leaf) {
        for (int i = 0; i <= sibling->num_keys; ++i) {
            child->children[child->num_keys + i] = sibling->children[i];
        }
    }

    // Update the total key count in the merged child.
    child->num_keys += sibling->num_keys;

    // 4. Shift keys and values in the parent to the left, as one key (the separator) has moved down.
    for (int i = idx + 1; i < node->num_keys; ++i) {
        node->keys[i - 1] = node->keys[i];
        node->values[i - 1] = node->values[i]; // If internal nodes store values.
    }
    // 5. Shift child pointers in the parent to the left, as one child (the sibling) has been merged.
    for (int i = idx + 2; i <= node->num_keys; ++i) {
        node->children[i - 1] = node->children[i];
    }

    node->num_keys--; // Parent loses one key.
    free(sibling);     // Free the merged sibling node to prevent memory leaks.
}

/**
 * @brief Ensures that a child node has at least ORDER keys before descending into it for deletion.
 * If the child is deficient (has ORDER-1 keys), it attempts to either borrow a key from a sibling
 * or merge with a sibling. This maintains the B-tree properties.
 * @param node Pointer to the parent node.
 * @param idx Index of the child node that needs to be filled/checked.
 */
void btree_fill(BTreeNode *node, int idx) {
    // Case 1: Try to borrow from the left sibling (if it exists and has enough keys)
    if (idx != 0 && node->children[idx - 1]->num_keys >= ORDER) {
        btree_borrow_from_prev(node, idx);
    }
    // Case 2: Else, try to borrow from the right sibling (if it exists and has enough keys)
    else if (idx != node->num_keys && node->children[idx + 1]->num_keys >= ORDER) {
        btree_borrow_from_next(node, idx);
    }
    // Case 3: Else (neither sibling can lend a key), merge with a sibling.
    // Prefer merging with the right sibling if it exists, otherwise merge with the left.
    else {
        if (idx != node->num_keys) {
            btree_merge(node, idx);
        } else { // Must merge with the left sibling if no right sibling or right sibling is deficient
            btree_merge(node, idx - 1);
        }
    }
}

/* ===================== B-Tree Deletion Core Logic ===================== */

/**
 * @brief Recursive function to remove a key from the B-Tree.
 * This function handles the complex logic of B-tree deletion, including:
 * 1. Removing keys from leaf nodes.
 * 2. Replacing keys in internal nodes with predecessors/successors and then deleting them from leaves.
 * 3. Handling underflow scenarios by borrowing keys from siblings or merging nodes.
 * @param node Pointer to the current node being processed.
 * @param key The key to be removed.
 * @return The current node (may be modified, or freed if it was the root and became empty).
 */
BTreeNode* btree_remove_recursive(BTreeNode *node, int key) {
    int idx = 0;
    // Find the first key in the current node that is greater than or equal to 'key'.
    // This 'idx' will either point to the key itself or the child pointer to follow.
    while (idx < node->num_keys && node->keys[idx] < key) {
        idx++;
    }

    // Scenario A: The key is found in the current node (`node->keys[idx] == key`).
    if (idx < node->num_keys && node->keys[idx] == key) {
        if (node->leaf) {
            // Case A.1: Key is found in a leaf node.
            // Simple removal: Shift all subsequent keys and values to the left by one position
            // to overwrite the key being removed and close the gap.
            for (int i = idx + 1; i < node->num_keys; ++i) {
                node->keys[i - 1] = node->keys[i];
                node->values[i - 1] = node->values[i];
            }
            node->num_keys--; // Decrement the count of keys in this node.
        } else {
            // Case A.2: Key is found in an internal node.
            // To remove an internal key, it must be replaced by its predecessor or successor,
            // and then that predecessor/successor is removed from its original leaf location.

            BTreeNode *child_pred = node->children[idx];      // The child to the left of the key
            BTreeNode *child_succ = node->children[idx + 1];  // The child to the right of the key

            // Subcase A.2.1: The left child (predecessor's subtree) has enough keys (at least ORDER).
            // We can borrow the predecessor from it.
            if (child_pred->num_keys >= ORDER) {
                int pred_key = btree_get_pred(node, idx); // Get the largest key from the left subtree (predecessor).
                node->keys[idx] = pred_key;               // Replace the current key with its predecessor.
                // Recursively delete the predecessor from its original location in the left child.
                node->children[idx] = btree_remove_recursive(node->children[idx], pred_key);
            }
            // Subcase A.2.2: Else, if the right child (successor's subtree) has enough keys.
            // We can borrow the successor from it.
            else if (child_succ->num_keys >= ORDER) {
                int succ_key = btree_get_succ(node, idx); // Get the smallest key from the right subtree (successor).
                node->keys[idx] = succ_key;               // Replace the current key with its successor.
                // Recursively delete the successor from its original location in the right child.
                node->children[idx + 1] = btree_remove_recursive(node->children[idx + 1], succ_key);
            }
            // Subcase A.2.3: Both children (left and right) are deficient (have ORDER-1) keys.
            // We must merge these two children. The key from the current node (node->keys[idx])
            // will also be pulled down into the merged node.
            else {
                // Merge the left child (`child_pred`) and the right child (`child_succ`),
                // effectively consuming `node->keys[idx]`.
                btree_merge(node, idx);
                // Now, recursively delete the key (which is now in the merged child) from the merged child.
                node->children[idx] = btree_remove_recursive(node->children[idx], key);
            }
        }
    }
    // Scenario B: The key is NOT found in the current node. We need to descend to a child.
    else {
        // If we've reached a leaf node and the key was not found, it means the key is not in the tree.
        if (node->leaf) {
            // printf("Key %d not found in B-tree.\n", key); // Commented to reduce excessive output
            return node; // No changes needed if key not found in leaf
        }

        // Determine if the child to descend into is the last child. This is important
        // because merging operations might change the indices of children.
        int is_last_child = (idx == node->num_keys) ? 1 : 0;

        BTreeNode *child = node->children[idx]; // The child pointer to follow.

        // Pre-check: If the child we are about to descend into is deficient (fewer than ORDER keys),
        // we must "fill" it first by borrowing from a sibling or merging with one.
        // This ensures that when we return from the recursive call, the child won't be
        // even more deficient (e.g., empty) and cause issues up the tree.
        if (child->num_keys < ORDER) {
            btree_fill(node, idx);
        }

        // After `btree_fill` (which might have merged `child` with a sibling),
        // re-evaluate which child to descend into. If `is_last_child` was true
        // and a merge occurred, the correct child might now be `idx - 1`.
        if (is_last_child && node->num_keys > idx) {
            // The original `idx` was the last child, and a merge occurred
            // (likely with the left sibling, making `idx-1` the new target).
            node->children[idx - 1] = btree_remove_recursive(node->children[idx - 1], key);
        } else {
            // Otherwise, descend into the child at `idx`.
            node->children[idx] = btree_remove_recursive(node->children[idx], key);
        }
    }

    return node; // Return the (potentially modified) current node.
}

/* ===================== B-Tree Public Remove Function ===================== */

/**
 * @brief Public interface to remove a key from the B-Tree.
 * This function serves as the entry point for deletion and handles the
 * special case where the root of the tree might become empty after a deletion,
 * in which case its only child becomes the new root.
 * @param root A pointer to the pointer to the root node of the B-Tree.
 * This allows the root pointer in `main` to be updated.
 * @param key The key to be removed.
 */
void btree_remove(BTreeNode **root, int key) {
    if (!(*root)) {
        //printf("Tree is empty. Cannot remove %d.\n", key); // Commented to reduce excessive output
        return;
    }

    // Call the main recursive removal function.
    // The recursive function returns the (potentially modified) root node.
    *root = btree_remove_recursive(*root, key);

    // After deletion, check if the root has become empty (0 keys) and is not a leaf.
    // This typically happens when the root had only one child, and a merge operation
    // caused that child to become the new root.
    if ((*root) && (*root)->num_keys == 0 && !(*root)->leaf) {
        BTreeNode *temp = *root;           // Store the current empty root.
        *root = (*root)->children[0];      // Its single child becomes the new root.
        free(temp);                        // Free the old empty root node to prevent memory leaks.
    }
}

/* ===================== B-Tree Memory Cleanup ===================== */

/**
 * @brief Recursively frees all nodes in the B-tree.
 * This is crucial to prevent memory leaks, especially for large trees.
 * @param node Pointer to the current node to free.
 */
void btree_destroy(BTreeNode *node) {
    if (node == NULL) {
        return;
    }
    if (!node->leaf) {
        for (int i = 0; i <= node->num_keys; ++i) {
            btree_destroy(node->children[i]);
        }
    }
    free(node);
}

/* ===================== Dynamic List Helper Functions (for parsing) ===================== */

void resize_int_array(int **arr, long long *count, long long *capacity, long long min_needed_capacity) {
    if (*count >= *capacity) {
        long long new_cap = (*capacity == 0) ? NUM_OPERATIONS : *capacity * 2;
        if (new_cap < min_needed_capacity) {
             new_cap = min_needed_capacity;
        }
        int *new_arr = realloc(*arr, (size_t)new_cap * sizeof(int));
        if (!new_arr) {
            perror("Failed to reallocate integer array");
            exit(EXIT_FAILURE);
        }
        *arr = new_arr;
        *capacity = new_cap;
    }
}

void resize_insert_op_array(InsertOp **arr, long long *count, long long *capacity, long long min_needed_capacity) {
    if (*count >= *capacity) {
        long long new_cap = (*capacity == 0) ? NUM_OPERATIONS : *capacity * 2;
        if (new_cap < min_needed_capacity) {
             new_cap = min_needed_capacity;
        }
        InsertOp *new_arr = realloc(*arr, (size_t)new_cap * sizeof(InsertOp));
        if (!new_arr) {
            perror("Failed to reallocate InsertOp array");
            exit(EXIT_FAILURE);
        }
        *arr = new_arr;
        *capacity = new_cap;
    }
}

void add_insert_op(int key, int value) {
    resize_insert_op_array(&insert_ops_list, &insert_ops_count, &insert_ops_capacity, insert_ops_count + 1);
    insert_ops_list[insert_ops_count].key = key;
    insert_ops_list[insert_ops_count].value = value;
    insert_ops_count++;
}

void add_lookup_key(int key) {
    resize_int_array(&lookup_keys_list, &lookup_keys_count, &lookup_keys_capacity, lookup_keys_count + 1);
    lookup_keys_list[lookup_keys_count++] = key;
}

void add_lowerbound_key(int key) {
    resize_int_array(&lowerbound_keys_list, &lowerbound_keys_count, &lowerbound_keys_capacity, lowerbound_keys_count + 1);
    lowerbound_keys_list[lowerbound_keys_count++] = key;
}

void add_remove_key(int key) {
    resize_int_array(&remove_keys_list, &remove_keys_count, &remove_keys_capacity, remove_keys_count + 1);
    remove_keys_list[remove_keys_count++] = key;
}

void parse_and_collect_operations(const char *filename) {
    FILE *fp = fopen(filename, "r");
    if (!fp) {
        fprintf(stderr, "ERROR: Failed to open file %s. Please ensure it exists and is accessible.\n", filename);
        exit(EXIT_FAILURE);
    }

    char line[LINE_LEN];
    long long line_num = 0;
    printf("Parsing and collecting operations from file: %s for B-tree...\n", filename);

    while (fgets(line, sizeof(line), fp)) {
        line_num++;
        line[strcspn(line, "\r\n")] = 0;

        if (strncmp(line, "insert", 6) == 0) {
            int k, v;
            if (sscanf(line, "insert %d %d", &k, &v) == 2) {
                add_insert_op(k, v);
                if (!fgets(line, sizeof(line), fp)) break; // Consume 'ok' line
            } else {
                fprintf(stderr, "ERROR: Malformed insert line on line %lld: '%s'\n", line_num, line);
            }
        } else if (strncmp(line, "lookup", 6) == 0) {
            int k;
            if (sscanf(line, "lookup %d", &k) == 1) {
                add_lookup_key(k);
                if (!fgets(line, sizeof(line), fp)) break; // Consume result line
            } else {
                fprintf(stderr, "ERROR: Malformed lookup line on line %lld: '%s'\n", line_num, line);
            }
        } else if (strncmp(line, "remove", 6) == 0) {
            int k;
            if (sscanf(line, "remove %d", &k) == 1) {
                add_remove_key(k);
                if (!fgets(line, sizeof(line), fp)) break; // Consume 'ok' line
            } else {
                fprintf(stderr, "ERROR: Malformed remove line on line %lld: '%s'\n", line_num, line);
            }
        } else if (strncmp(line, "lowerbound", 10) == 0) {
            int k;
            if (sscanf(line, "lowerbound %d", &k) == 1) {
                add_lowerbound_key(k);
                if (!fgets(line, sizeof(line), fp)) break; // Consume result line
            } else {
                fprintf(stderr, "ERROR: Malformed lowerbound line on line %lld: '%s'\n", line_num, line);
            }
        } else if (strlen(line) > 0 && line[0] != '\n' && line[0] != '\r') {
            //fprintf(stderr, "WARNING: Unrecognized or invalid command on line %lld: '%s'\n", line_num, line);
        }
    }
    fclose(fp);
    printf("Finished parsing for B-tree. Collected %lld inserts, %lld lookups, %lld lowerbounds, %lld removals.\n",
           insert_ops_count, lookup_keys_count, lowerbound_keys_count, remove_keys_count);
}

// Helper function to calculate time difference in seconds
double get_elapsed_time(struct timespec start, struct timespec end) {
    return (double)(end.tv_sec - start.tv_sec) + (double)(end.tv_nsec - start.tv_nsec) / 1e9;
}

/* ===================== Main Function ===================== */

int main(int argc, char *argv[]) {
    srand((unsigned int)time(NULL));

    if (argc != 2) {
        printf("Usage: %s <benchmark_operations_file>\n", argv[0]);
        return 1;
    }

    printf("--- Initializing B-tree Data Structure ---\n");
    BTreeNode *btree = create_btree_node(1); // Initialize B-tree root as a leaf

    parse_and_collect_operations(argv[1]);

    printf("\n--- Starting B-tree Benchmarks ---\n");
    struct timespec start_total, end_total;
    clock_gettime(CLOCK_MONOTONIC, &start_total);

    // Insert Benchmark
    printf("\nAttempting to run B-tree Insert Benchmark (%lld operations)...\n", insert_ops_count);
    struct timespec start_insert, end_insert;
    clock_gettime(CLOCK_MONOTONIC, &start_insert);
    for (long long i = 0; i < insert_ops_count; ++i) {
        btree_insert(&btree, insert_ops_list[i].key, insert_ops_list[i].value);
    }
    clock_gettime(CLOCK_MONOTONIC, &end_insert);
    printf("B-tree Insert time: %.6f seconds\n", get_elapsed_time(start_insert, end_insert));
    printf("B-tree Insert Benchmark function call completed.\n");

    // Lookup Benchmark
    printf("\nAttempting to run B-tree Lookup Benchmark (%lld operations)...\n", lookup_keys_count);
    struct timespec start_lookup, end_lookup;
    clock_gettime(CLOCK_MONOTONIC, &start_lookup);
    for (long long i = 0; i < lookup_keys_count; ++i) {
        btree_lookup(btree, lookup_keys_list[i]);
    }
    clock_gettime(CLOCK_MONOTONIC, &end_lookup);
    printf("B-tree Lookup time: %.6f seconds\n", get_elapsed_time(start_lookup, end_lookup));
    printf("B-tree Lookup Benchmark function call completed.\n");

    // Lowerbound Benchmark
    printf("\nAttempting to run B-tree Lowerbound Benchmark (%lld operations)...\n", lowerbound_keys_count);
    struct timespec start_lowerbound, end_lowerbound;
    clock_gettime(CLOCK_MONOTONIC, &start_lowerbound);
    for (long long i = 0; i < lowerbound_keys_count; ++i) {
        btree_lowerbound(btree, lowerbound_keys_list[i]);
    }
    clock_gettime(CLOCK_MONOTONIC, &end_lowerbound);
    printf("B-tree Lowerbound time: %.6f seconds\n", get_elapsed_time(start_lowerbound, end_lowerbound));
    printf("B-tree Lowerbound Benchmark function call completed.\n");

    // Remove Benchmark
    printf("\nAttempting to run B-tree Remove Benchmark (%lld operations)...\n", remove_keys_count);
    struct timespec start_remove, end_remove;
    clock_gettime(CLOCK_MONOTONIC, &start_remove);
    for (long long i = 0; i < remove_keys_count; ++i) {
        btree_remove(&btree, remove_keys_list[i]);
    }
    clock_gettime(CLOCK_MONOTONIC, &end_remove);
    printf("B-tree Remove time: %.6f seconds\n", get_elapsed_time(start_remove, end_remove));
    printf("B-tree Remove Benchmark function call completed.\n");

    clock_gettime(CLOCK_MONOTONIC, &end_total);
    printf("\nTotal B-tree Benchmark time: %.6f seconds\n", get_elapsed_time(start_total, end_total));

    printf("\n--- Starting B-tree Memory Cleanup ---\n");
    if (insert_ops_list) { free(insert_ops_list); insert_ops_list = NULL; }
    if (lookup_keys_list) { free(lookup_keys_list); lookup_keys_list = NULL; }
    if (lowerbound_keys_list) { free(lowerbound_keys_list); lowerbound_keys_list = NULL; }
    if (remove_keys_list) { free(remove_keys_list); remove_keys_list = NULL; }
    if (btree) {
        btree_destroy(btree);
        btree = NULL;
    }
    printf("B-tree Memory cleanup completed.\n");
    printf("\nAll B-tree benchmarks and memory cleanup completed.\n");

    return 0;
}


Writing btree.c


In [ ]:
!gcc btree.c -o btree

In [ ]:
!./btree combined_benchmark_200k_ops.txt

--- Initializing B-tree Data Structure ---
Parsing and collecting operations from file: combined_benchmark_200k_ops.txt for B-tree...
Finished parsing for B-tree. Collected 100000 inserts, 100000 lookups, 100000 lowerbounds, 100000 removals.

--- Starting B-tree Benchmarks ---

Attempting to run B-tree Insert Benchmark (100000 operations)...
B-tree Insert time: 0.028063 seconds
B-tree Insert Benchmark function call completed.

Attempting to run B-tree Lookup Benchmark (100000 operations)...
B-tree Lookup time: 0.023367 seconds
B-tree Lookup Benchmark function call completed.

Attempting to run B-tree Lowerbound Benchmark (100000 operations)...
B-tree Lowerbound time: 0.024477 seconds
B-tree Lowerbound Benchmark function call completed.

Attempting to run B-tree Remove Benchmark (100000 operations)...
B-tree Remove time: 0.039110 seconds
B-tree Remove Benchmark function call completed.

Total B-tree Benchmark time: 0.115150 seconds

--- Starting B-tree Memory Cleanup ---
B-tree Memory c

In [ ]:
!./bree /content/combined_benchmark_2m_ops.txt

/bin/bash: line 1: ./bree: No such file or directory


## Combined Alex and BTree code

In [1]:
%%writefile alex_btree_combined_benchmark.c
/*
 * alex_btree_combined_benchmark.c (ALEX and B-tree Combined Benchmark)
 * ---------------------------------------------------------------------
 * This program implements and benchmarks both the ALEX (Adaptive Learned Index)
 * and a B+Tree data structure for comparison, based on concepts from the paper
 * "ALEX: An Updatable Adaptive Learned Index".
 *
 * It reads commands (insert, lookup, lowerbound, remove) from a single input file,
 * categorizes them, and then runs separate benchmark phases for each operation type
 * for both ALEX and B-tree, reporting individual execution times.
 *
 * This version is configured for 1,000,000 operations per benchmark phase.
 * It is adapted for Linux/POSIX environments (e.g., Google Colab) using
 * clock_gettime for high-resolution timing.
 */

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>     // For time() to seed random number generator and clock_gettime

// Define the number of operations for each benchmark phase
#define NUM_OPERATIONS 1000000

// LEAF_CAPACITY for ALEX is set to NUM_OPERATIONS for this simplified benchmark.
// In a real ALEX, this would dynamically adjust based on data distribution.
#define LEAF_CAPACITY NUM_OPERATIONS

// Order of the B+Tree. A common value for in-memory B-trees is 32 or 64.
// This means internal nodes can have between ORDER-1 and 2*ORDER-1 keys,
// and between ORDER and 2*ORDER children.
#define ORDER 32

// Maximum length of a line when reading from the input file
#define LINE_LEN 128

/* ===================== Global Operation Lists for Collection ===================== */
// These global arrays store the operations parsed from the input file,
// allowing us to run the same set of operations on both ALEX and B-tree.

// Structure to hold an insert operation's key and value
typedef struct {
    int key;
    int value;
} InsertOp;

// Dynamic arrays to store all operations parsed from the input file
InsertOp *insert_ops_list = NULL;
long long insert_ops_count = 0;
long long insert_ops_capacity = 0;

int *lookup_keys_list = NULL;
long long lookup_keys_count = 0;
long long lookup_keys_capacity = 0;

int *lowerbound_keys_list = NULL;
long long lowerbound_keys_count = 0;
long long lowerbound_keys_capacity = 0;

int *remove_keys_list = NULL;
long long remove_keys_count = 0;
long long remove_keys_capacity = 0;

/* ===================== ALEX Structures and Functions ===================== */
// ALEX uses a learned model (linear regression in this simplified version)
// to predict the position of a key within a sorted array (leaf node).

// Simple linear model for predicting key position in a leaf
typedef struct {
    float a, b; // Coefficients for y = ax + b
} LinearModel;

// Leaf node for ALEX index
typedef struct {
    int *keys;              // Pointer to dynamically allocated array of keys
    int *values;            // Pointer to dynamically allocated array of values
    int size;               // Current number of keys stored in this leaf
    int capacity;           // Actual allocated capacity for keys and values arrays
    LinearModel model;      // Model to predict key position within this leaf
} ALEXLeaf;

/**
 * @brief Initializes an ALEXLeaf structure by allocating memory for its internal arrays.
 * @param leaf Pointer to the ALEXLeaf structure to initialize.
 * @param initial_capacity The desired initial capacity for the keys and values arrays.
 */
void alex_init(ALEXLeaf *leaf, int initial_capacity) {
    // printf("Attempting to allocate ALEXLeaf with capacity: %d\n", initial_capacity); // Debug print
    leaf->keys = (int *)malloc((size_t)initial_capacity * sizeof(int));
    leaf->values = (int *)malloc((size_t)initial_capacity * sizeof(int));
    if (!leaf->keys || !leaf->values) {
        perror("Failed to allocate memory for ALEXLeaf keys or values");
        free(leaf->keys);   // Free already allocated if one fails
        free(leaf->values); // Safe to call free(NULL)
        exit(EXIT_FAILURE);
    }
    leaf->size = 0;
    leaf->capacity = initial_capacity;
    leaf->model.a = 0.0;
    leaf->model.b = 0.0;
    // printf("ALEXLeaf initialized with capacity: %d\n", initial_capacity); // Debug print
}

/**
 * @brief Deallocates memory associated with an ALEXLeaf structure.
 * @param leaf Pointer to the ALEXLeaf structure to destroy.
 */
void alex_destroy(ALEXLeaf *leaf) {
    if (leaf) {
        // printf("Destroying ALEXLeaf data at address %p\n", (void*)leaf->keys); // Debug print
        free(leaf->keys);
        free(leaf->values);
        leaf->keys = NULL;
        leaf->values = NULL;
        leaf->size = 0;
        leaf->capacity = 0;
    }
}

/**
 * @brief Retrains the linear model based on the current set of keys in the leaf.
 * This simple linear model assumes keys are sorted and maps them to indices.
 * @param model Pointer to the LinearModel to train.
 * @param keys Array of keys in the leaf.
 * @param n Number of keys in the array.
 */
void train_model(LinearModel *model, int *keys, int n) {
    if (n == 0) {
        model->a = 0.0;
        model->b = 0.0;
        return;
    }
    if (n == 1) { // Special case for single element to avoid division by zero or nonsensical model
        model->a = 0.0;
        model->b = 0.0; // Predicts index 0
        return;
    }
    int min_x = keys[0];        // Smallest key
    int max_x = keys[n - 1];    // Largest key (assuming sorted)

    if (max_x == min_x) { // All keys are identical
        model->a = 0.0;
        model->b = (n - 1) / 2.0; // Predicts middle index
    } else {
        // Calculate slope (a) and y-intercept (b) for linear regression
        // y = index, x = key. We want to predict index from key.
        model->a = (float)(n - 1) / (max_x - min_x);
        model->b = -model->a * min_x;
    }
}

/**
 * @brief Predicts the approximate position of a key using the linear model.
 * @param model Pointer to the LinearModel.
 * @param key The key for which to predict the position.
 * @return The predicted index.
 */
int predict(LinearModel *model, int key) {
    return (int)(model->a * key + model->b);
}

/**
 * @brief Inserts a key-value pair into an ALEXLeaf.
 * The key is inserted at the predicted position, and then shifted if necessary
 * to maintain sorted order. The model is retrained after insertion.
 * @param leaf Pointer to the ALEXLeaf.
 * @param key The key to insert.
 * @param value The value to associate with the key.
 */
void alex_insert(ALEXLeaf *leaf, int key, int value) {
    if (leaf->size >= leaf->capacity) {
        // In this simplified ALEX, the leaf has a fixed capacity.
        // A full ALEX would split this leaf into multiple leaves and update the tree structure.
        fprintf(stderr, "ALEXLeaf full (key %d). Current size: %d, Max capacity: %d. Further inserts will be ignored.\n", key, leaf->size, leaf->capacity);
        return;
    }

    int pos = predict(&leaf->model, key); // Get predicted position
    // Adjust predicted position to be within valid bounds [0, size]
    if (pos < 0) pos = 0;
    if (pos > leaf->size) pos = leaf->size;

    // Correct position by linear scan (or exponential/binary search in a real ALEX)
    // Shift elements to the right to make space for the new key
    while (pos < leaf->size && leaf->keys[pos] < key) pos++;
    for (int i = leaf->size; i > pos; i--) {
        leaf->keys[i] = leaf->keys[i - 1];
        leaf->values[i] = leaf->values[i - 1];
    }
    // Insert the new key-value pair
    leaf->keys[pos] = key;
    leaf->values[pos] = value;
    leaf->size++;
    // Retrain the model to reflect the new data distribution
    train_model(&leaf->model, leaf->keys, leaf->size);
}

/**
 * @brief Looks up a key in an ALEXLeaf.
 * For simplicity, this uses a linear scan. A real ALEX would use the model
 * prediction and then a more efficient search (e.g., exponential search)
 * within a small error bound.
 * @param leaf Pointer to the ALEXLeaf.
 * @param key The key to lookup.
 * @return The value associated with the key, or -1 if not found.
 */
int alex_lookup(ALEXLeaf *leaf, int key) {
    for (int i = 0; i < leaf->size; i++) {
        if (leaf->keys[i] == key) return leaf->values[i];
    }
    return -1; // Key not found
}

/**
 * @brief Removes a key from an ALEXLeaf.
 * This performs a linear scan to find the key and then shifts elements
 * to fill the gap. The model is retrained after removal.
 * @param leaf Pointer to the ALEXLeaf.
 * @param key The key to remove.
 */
void alex_remove(ALEXLeaf *leaf, int key) {
    for (int i = 0; i < leaf->size; i++) {
        if (leaf->keys[i] == key) {
            // Shift elements to the left to overwrite the removed key
            for (int j = i; j < leaf->size - 1; j++) {
                leaf->keys[j] = leaf->keys[j + 1];
                leaf->values[j] = leaf->values[j + 1];
            }
            leaf->size--;
            train_model(&leaf->model, leaf->keys, leaf->size); // Retrain model
            break; // Key found and removed, exit loop
        }
    }
}

/**
 * @brief Finds the smallest key in an ALEXLeaf that is greater than or equal to the given key.
 * This implements a simple linear scan.
 * @param leaf Pointer to the ALEXLeaf.
 * @param key The key for which to find the lower bound.
 * @return The lower bound key, or -1 if no such key exists.
 */
int alex_lowerbound(ALEXLeaf *leaf, int key) {
    for (int i = 0; i < leaf->size; i++) {
        if (leaf->keys[i] >= key) return leaf->keys[i];
    }
    return -1; // No key found >= target key
}

/* ===================== B+Tree Structures and Functions ===================== */
// A traditional B+Tree implementation for comparison.

typedef struct BTreeNode {
    int keys[2 * ORDER - 1];          // Max (2*ORDER - 1) keys
    int values[2 * ORDER - 1];        // Values associated with keys (only in leaf nodes for B+ trees)
    struct BTreeNode *children[2 * ORDER]; // Max (2*ORDER) children pointers (for internal nodes)
    int leaf;                         // 1 if leaf node, 0 if internal node
    int num_keys;                     // Current number of keys in the node
} BTreeNode;

/**
 * @brief Creates and initializes a new B+Tree node.
 * @param leaf 1 if it's a leaf node, 0 if an internal node.
 * @return A pointer to the newly created node.
 */
BTreeNode* create_btree_node(int leaf) {
    BTreeNode *node = (BTreeNode *)malloc(sizeof(BTreeNode));
    if (!node) {
        perror("Failed to allocate BTreeNode");
        exit(EXIT_FAILURE);
    }
    node->leaf = leaf;
    node->num_keys = 0;
    for (int i = 0; i < 2 * ORDER; i++)
        node->children[i] = NULL;
    return node;
}

// Forward declarations for recursive B-tree functions
void btree_split_child(BTreeNode *parent, int i);
void btree_insert_nonfull(BTreeNode *node, int key, int value);

/**
 * @brief Inserts a key-value pair into the B+Tree.
 * Handles the case where the root needs to be split.
 * @param root A pointer to the pointer to the root node (allows updating the root).
 * @param key The key to insert.
 * @param value The value to insert.
 */
void btree_insert(BTreeNode **root, int key, int value) {
    BTreeNode *r = *root;
    // If the root node is full, the tree grows in height
    if (r->num_keys == 2 * ORDER - 1) {
        BTreeNode *s = create_btree_node(0); // Create a new root (internal node)
        s->children[0] = r;                  // Old root becomes the first child of the new root
        btree_split_child(s, 0);             // Split the old root
        *root = s;                           // Update the global root pointer
        btree_insert_nonfull(s, key, value); // Insert the key into the new root's children
    } else {
        // If the root is not full, just insert into it
        btree_insert_nonfull(r, key, value);
    }
}

/**
 * @brief Inserts a key-value pair into a non-full B-tree node.
 * This is a recursive helper function for `btree_insert`.
 * @param node Pointer to the current node (guaranteed not to be full).
 * @param key The key to insert.
 * @param value The value to insert.
 */
void btree_insert_nonfull(BTreeNode *node, int key, int value) {
    int i = node->num_keys - 1; // Start from the rightmost key position

    if (node->leaf) {
        // If it's a leaf node, find the correct position and insert
        while (i >= 0 && key < node->keys[i]) {
            node->keys[i + 1] = node->keys[i];       // Shift keys
            node->values[i + 1] = node->values[i];   // Shift values
            i--;
        }
        node->keys[i + 1] = key;   // Insert the new key
        node->values[i + 1] = value; // Insert the new value
        node->num_keys++;          // Increment key count
    } else {
        // If it's an internal node, find the appropriate child to descend into
        while (i >= 0 && key < node->keys[i]) i--;
        i++; // 'i' is now the index of the child pointer

        // If the child is full, split it before descending
        if (node->children[i]->num_keys == 2 * ORDER - 1) {
            btree_split_child(node, i); // Split the child
            // After splitting, a new key might have been promoted to 'node',
            // and we need to determine which of the two new children to descend into.
            if (key > node->keys[i]) i++;
        }
        // Recursively call insert on the appropriate child
        btree_insert_nonfull(node->children[i], key, value);
    }
}

/**
 * @brief Splits a full child of a B-tree node.
 * This function assumes `parent->children[i]` (let's call it `y`) is full (2*ORDER - 1 keys).
 * It splits `y` into two nodes (`y` and `z`), moves the median key to `parent`,
 * and adjusts parent's children and key counts.
 *
 * @param parent Pointer to the parent node.
 * @param i Index of the child that needs to be split.
 */
void btree_split_child(BTreeNode *parent, int i) {
    BTreeNode *y = parent->children[i];         // The child to be split (full node)
    BTreeNode *z = create_btree_node(y->leaf); // Create a new node for the right half

    // Copy the upper half of keys (from median+1 to end) from y to z.
    // In a B-tree with max 2*ORDER-1 keys, the median key is at index ORDER-1.
    // Keys moved to z start from index ORDER in y.
    for (int j = 0; j < ORDER - 1; j++) {
        z->keys[j] = y->keys[j + ORDER];
        z->values[j] = y->values[j + ORDER]; // Copy values if they are stored in internal nodes (B+tree only in leaves)
    }

    // If y is an internal node, copy the upper half of its children pointers to z.
    // Children moved to z start from index ORDER in y.
    if (!y->leaf) {
        for (int j = 0; j < ORDER; j++) {
            z->children[j] = y->children[j + ORDER];
        }
    }

    // Set the number of keys for the new node z. It receives ORDER-1 keys.
    z->num_keys = ORDER - 1;

    // Update the number of keys for the original node y. It retains ORDER-1 keys.
    y->num_keys = ORDER - 1;

    // Shift children pointers in the parent to make space for the new node z.
    // This makes space at parent->children[i+1]
    for (int j = parent->num_keys; j >= i + 1; j--)
        parent->children[j + 1] = parent->children[j];
    parent->children[i + 1] = z; // Assign the new node z as a child of parent

    // Shift keys and values in the parent to make space for the median key.
    // This makes space at parent->keys[i]
    for (int j = parent->num_keys - 1; j >= i; j--) {
        parent->keys[j + 1] = parent->keys[j];
        parent->values[j + 1] = parent->values[j]; // Values also shifted if present
    }
    // Move the median key and its value from y to the parent.
    // The median key is at y->keys[ORDER - 1].
    parent->keys[i] = y->keys[ORDER - 1];
    parent->values[i] = y->values[ORDER - 1];

    parent->num_keys++; // Parent gains one key
}

/**
 * @brief Looks up a key in the B+Tree.
 * This is a recursive function that traverses the tree.
 * @param node Pointer to the current node being searched.
 * @param key The key to lookup.
 * @return The value associated with the key, or -1 if not found.
 */
int btree_lookup(BTreeNode *node, int key) {
    int i = 0;
    // Find the first key in the current node that is greater than or equal to the target key.
    while (i < node->num_keys && key > node->keys[i]) i++;

    // If key is found in this node
    if (i < node->num_keys && key == node->keys[i]) return node->values[i];

    // If it's a leaf node and key not found, return -1
    if (node->leaf) return -1;

    // If it's an internal node, descend into the appropriate child
    return btree_lookup(node->children[i], key);
}

/**
 * @brief Finds the smallest key in the B+Tree that is greater than or equal to the given key.
 * This is a recursive function that traverses the tree.
 * @param node Pointer to the current node being searched.
 * @param key The key for which to find the lower bound.
 * @return The lower bound key, or -1 if no such key exists.
 */
int btree_lowerbound(BTreeNode *node, int key) {
    int i = 0;
    // Find the first key in the current node that is greater than or equal to the target key.
    // Or find the appropriate child pointer to descend into.
    while (i < node->num_keys && key > node->keys[i]) {
        i++;
    }

    // If 'i' is within the bounds of keys and the key at 'i' is >= target key
    if (i < node->num_keys && node->keys[i] >= key) {
        if (node->leaf) {
            // If it's a leaf node, this is the first key >= target, so it's our lower bound.
            return node->keys[i];
        } else {
            // If it's an internal node, the lower bound could be in the child pointed to by children[i]
            // or it could be the key at node->keys[i] itself.
            int result_in_child = btree_lowerbound(node->children[i], key);
            if (result_in_child != -1) {
                // If a lower bound was found in the child, that's our result.
                return result_in_child;
            } else {
                // If no lower bound found in the child, it means all keys in that child's subtree
                // are less than 'key'. So, the current key at node->keys[i] is the smallest
                // key in this subtree that is greater than or equal to 'key'.
                return node->keys[i];
            }
        }
    } else {
        // If 'key' is greater than all keys in this node, and we are an internal node,
        // we must check the last child (node->children[node->num_keys]) for the lower bound.
        if (!node->leaf) {
            return btree_lowerbound(node->children[node->num_keys], key);
        } else {
            // If it's a leaf node and 'key' is greater than all its keys,
            // then no key greater than or equal to 'key' exists in this leaf or its subtree.
            return -1;
        }
    }
}


// Forward declarations for B-tree removal functions
BTreeNode* btree_remove_recursive(BTreeNode *node, int key);
int btree_get_pred(BTreeNode *node, int idx);
int btree_get_succ(BTreeNode *node, int idx);
void btree_fill(BTreeNode *node, int idx);
void btree_borrow_from_prev(BTreeNode *node, int idx);
void btree_borrow_from_next(BTreeNode *node, int idx);
void btree_merge(BTreeNode *node, int idx);


/* ===================== B-Tree Deletion Helper Functions ===================== */

/**
 * @brief Finds the predecessor key for a key at a given index in an internal node.
 * The predecessor is the largest key in the left subtree of the key being deleted.
 * This is found by traversing to the rightmost leaf of the left child.
 * @param node Pointer to the parent node containing the key.
 * @param idx The index of the key in `node->keys` for which to find the predecessor.
 * @return The predecessor key.
 */
int btree_get_pred(BTreeNode *node, int idx) {
    // Start from the child node to the left of the key (node->children[idx])
    BTreeNode *curr = node->children[idx];
    // Traverse down to the rightmost leaf node in this subtree
    while (!curr->leaf) {
        curr = curr->children[curr->num_keys];
    }
    // The predecessor is the last key in this leaf node
    return curr->keys[curr->num_keys - 1];
}

/**
 * @brief Finds the successor key for a key at a given index in an internal node.
 * The successor is the smallest key in the right subtree of the key being deleted.
 * This is found by traversing to the leftmost leaf of the right child.
 * @param node Pointer to the parent node containing the key.
 * @param idx The index of the key in `node->keys` for which to find the successor.
 * @return The successor key.
 */
int btree_get_succ(BTreeNode *node, int idx) {
    // Start from the child node to the right of the key (node->children[idx + 1])
    BTreeNode *curr = node->children[idx + 1];
    // Traverse down to the leftmost leaf node in this subtree
    while (!curr->leaf) {
        curr = curr->children[0];
    }
    // The successor is the first key in this leaf node
    return curr->keys[0];
}

/**
 * @brief Borrows a key from the left sibling (previous sibling) of child 'idx'.
 * This operation is performed when `node->children[idx]` is deficient (has `ORDER-1` keys)
 * and its left sibling (`node->children[idx-1]`) has at least `ORDER` keys.
 * A key moves from the sibling to the parent, and a key from the parent moves to the deficient child.
 * @param node Pointer to the parent node.
 * @param idx Index of the child that needs a key.
 */
void btree_borrow_from_prev(BTreeNode *node, int idx) {
    BTreeNode *child = node->children[idx];        // The child that is deficient
    BTreeNode *sibling = node->children[idx - 1]; // The generous left sibling

    // 1. Shift all keys and values in the deficient child one position to the right
    //    to make space for the incoming key from the parent.
    for (int i = child->num_keys - 1; i >= 0; --i) {
        child->keys[i + 1] = child->keys[i];
        child->values[i + 1] = child->values[i];
    }

    // 2. If the deficient child is an internal node, shift its children pointers too.
    if (!child->leaf) {
        for (int i = child->num_keys; i >= 0; --i) {
            child->children[i + 1] = child->children[i];
        }
    }

    // 3. Move the separating key from the parent (node->keys[idx-1]) down to the
    //    deficient child's first position (index 0).
    child->keys[0] = node->keys[idx - 1];
    child->values[0] = node->values[idx - 1]; // If internal nodes store values, copy it too.

    // 4. If the deficient child is internal, move the rightmost child pointer
    //    from the sibling to the deficient child's first child pointer.
    if (!child->leaf) {
        child->children[0] = sibling->children[sibling->num_keys];
    }

    // 5. Move the rightmost key from the sibling (sibling->keys[sibling->num_keys - 1])
    //    up to the parent, replacing the key that moved down.
    node->keys[idx - 1] = sibling->keys[sibling->num_keys - 1];
    node->values[idx - 1] = sibling->values[sibling->num_keys - 1]; // Copy value too.

    child->num_keys++;    // Deficient child gains a key
    sibling->num_keys--;  // Sibling loses a key
}

/**
 * @brief Borrows a key from the right sibling (next sibling) of child 'idx'.
 * This operation is performed when `node->children[idx]` is deficient (has `ORDER-1` keys)
 * and its right sibling (`node->children[idx+1]`) has at least `ORDER` keys.
 * A key moves from the sibling to the parent, and a key from the parent moves to the deficient child.
 * @param node Pointer to the parent node.
 * @param idx Index of the child that needs a key.
 */
void btree_borrow_from_next(BTreeNode *node, int idx) {
    BTreeNode *child = node->children[idx];        // The child that is deficient
    BTreeNode *sibling = node->children[idx + 1]; // The generous right sibling

    // 1. Move the separating key from the parent (node->keys[idx]) down to the
    //    deficient child's last position.
    child->keys[child->num_keys] = node->keys[idx];
    child->values[child->num_keys] = node->values[idx]; // If internal nodes store values.

    // 2. If the deficient child is internal, move the leftmost child pointer
    //    from the sibling to the deficient child's last child pointer.
    if (!child->leaf) {
        child->children[child->num_keys + 1] = sibling->children[0];
    }

    // 3. Move the leftmost key from the sibling (sibling->keys[0])
    //    up to the parent, replacing the key that moved down.
    node->keys[idx] = sibling->keys[0];
    node->values[idx] = sibling->values[0]; // Copy value too.

    // 4. Shift all keys and values in the sibling one position to the left,
    //    to fill the gap created by moving its first key.
    for (int i = 1; i < sibling->num_keys; ++i) {
        sibling->keys[i - 1] = sibling->keys[i];
        sibling->values[i - 1] = sibling->values[i];
    }
    // 5. If the sibling is an internal node, shift its children pointers too.
    if (!sibling->leaf) {
        for (int i = 1; i <= sibling->num_keys; ++i) {
            sibling->children[i - 1] = sibling->children[i];
        }
    }

    child->num_keys++;    // Deficient child gains a key
    sibling->num_keys--;  // Sibling loses a key
}

/**
 * @brief Merges child node[idx] with its right sibling (child node[idx+1]).
 * This operation is performed when a child is deficient and neither sibling
 * can lend a key (both have `ORDER-1` keys).
 * The separating key from the parent (`node->keys[idx]`) is also pulled down
 * into the merged node. The right sibling is then freed.
 * @param node Pointer to the parent node.
 * @param idx Index of the left child involved in the merge.
 */
void btree_merge(BTreeNode *node, int idx) {
    BTreeNode *child = node->children[idx];        // The left child (will absorb the sibling)
    BTreeNode *sibling = node->children[idx + 1]; // The right sibling (will be merged into 'child')

    // 1. Pull the separating key from the parent (node->keys[idx]) down
    //    to the end of the left child's keys.
    child->keys[child->num_keys] = node->keys[idx];
    child->values[child->num_keys] = node->values[idx]; // If internal nodes store values.
    child->num_keys++; // Increment key count of the left child for the pulled-down key.

    // 2. Copy all keys and values from the right sibling to the left child.
    for (int i = 0; i < sibling->num_keys; ++i) {
        child->keys[child->num_keys + i] = sibling->keys[i];
        child->values[child->num_keys + i] = sibling->values[i]; // If internal nodes store values.
    }

    // 3. If the nodes are internal, copy all child pointers from the right sibling to the left child.
    if (!child->leaf) {
        for (int i = 0; i <= sibling->num_keys; ++i) {
            child->children[child->num_keys + i] = sibling->children[i];
        }
    }

    // Update the total key count in the merged child.
    child->num_keys += sibling->num_keys;

    // 4. Shift keys and values in the parent to the left, as one key (the separator) has moved down.
    for (int i = idx + 1; i < node->num_keys; ++i) {
        node->keys[i - 1] = node->keys[i];
        node->values[i - 1] = node->values[i]; // If internal nodes store values.
    }
    // 5. Shift child pointers in the parent to the left, as one child (the sibling) has been merged.
    for (int i = idx + 2; i <= node->num_keys; ++i) {
        node->children[i - 1] = node->children[i];
    }

    node->num_keys--; // Parent loses one key.
    free(sibling);     // Free the merged sibling node to prevent memory leaks.
}

/**
 * @brief Ensures that a child node has at least ORDER keys before descending into it for deletion.
 * If the child is deficient (has ORDER-1 keys), it attempts to either borrow a key from a sibling
 * or merge with a sibling. This maintains the B-tree properties.
 * @param node Pointer to the parent node.
 * @param idx Index of the child node that needs to be filled/checked.
 */
void btree_fill(BTreeNode *node, int idx) {
    // Case 1: Try to borrow from the left sibling (if it exists and has enough keys)
    if (idx != 0 && node->children[idx - 1]->num_keys >= ORDER) {
        btree_borrow_from_prev(node, idx);
    }
    // Case 2: Else, try to borrow from the right sibling (if it exists and has enough keys)
    else if (idx != node->num_keys && node->children[idx + 1]->num_keys >= ORDER) {
        btree_borrow_from_next(node, idx);
    }
    // Case 3: Else (neither sibling can lend a key), merge with a sibling.
    // Prefer merging with the right sibling if it exists, otherwise merge with the left.
    else {
        if (idx != node->num_keys) {
            btree_merge(node, idx);
        } else { // Must merge with the left sibling if no right sibling or right sibling is deficient
            btree_merge(node, idx - 1);
        }
    }
}

/* ===================== B-Tree Deletion Core Logic ===================== */

/**
 * @brief Recursive function to remove a key from the B-Tree.
 * This function handles the complex logic of B-tree deletion, including:
 * 1. Removing keys from leaf nodes.
 * 2. Replacing keys in internal nodes with predecessors/successors and then deleting them from leaves.
 * 3. Handling underflow scenarios by borrowing keys from siblings or merging nodes.
 * @param node Pointer to the current node being processed.
 * @param key The key to be removed.
 * @return The current node (may be modified, or freed if it was the root and became empty).
 */
BTreeNode* btree_remove_recursive(BTreeNode *node, int key) {
    int idx = 0;
    // Find the first key in the current node that is greater than or equal to 'key'.
    // This 'idx' will either point to the key itself or the child pointer to follow.
    while (idx < node->num_keys && node->keys[idx] < key) {
        idx++;
    }

    // Scenario A: The key is found in the current node (`node->keys[idx] == key`).
    if (idx < node->num_keys && node->keys[idx] == key) {
        if (node->leaf) {
            // Case A.1: Key is found in a leaf node.
            // Simple removal: Shift all subsequent keys and values to the left by one position
            // to overwrite the key being removed and close the gap.
            for (int i = idx + 1; i < node->num_keys; ++i) {
                node->keys[i - 1] = node->keys[i];
                node->values[i - 1] = node->values[i];
            }
            node->num_keys--; // Decrement the count of keys in this node.
        } else {
            // Case A.2: Key is found in an internal node.
            // To remove an internal key, it must be replaced by its predecessor or successor,
            // and then that predecessor/successor is removed from its original leaf location.

            BTreeNode *child_pred = node->children[idx];      // The child to the left of the key
            BTreeNode *child_succ = node->children[idx + 1];  // The child to the right of the key

            // Subcase A.2.1: The left child (predecessor's subtree) has enough keys (at least ORDER).
            // We can borrow the predecessor from it.
            if (child_pred->num_keys >= ORDER) {
                int pred_key = btree_get_pred(node, idx); // Get the largest key from the left subtree (predecessor).
                node->keys[idx] = pred_key;               // Replace the current key with its predecessor.
                // Recursively delete the predecessor from its original location in the left child.
                node->children[idx] = btree_remove_recursive(node->children[idx], pred_key);
            }
            // Subcase A.2.2: Else, if the right child (successor's subtree) has enough keys.
            // We can borrow the successor from it.
            else if (child_succ->num_keys >= ORDER) {
                int succ_key = btree_get_succ(node, idx); // Get the smallest key from the right subtree (successor).
                node->keys[idx] = succ_key;               // Replace the current key with its successor.
                // Recursively delete the successor from its original location in the right child.
                node->children[idx + 1] = btree_remove_recursive(node->children[idx + 1], succ_key);
            }
            // Subcase A.2.3: Both children (left and right) are deficient (have ORDER-1) keys.
            // We must merge these two children. The key from the current node (node->keys[idx])
            // will also be pulled down into the merged node.
            else {
                // Merge the left child (`child_pred`) and the right child (`child_succ`),
                // effectively consuming `node->keys[idx]`.
                btree_merge(node, idx);
                // Now, recursively delete the key (which is now in the merged child) from the merged child.
                node->children[idx] = btree_remove_recursive(node->children[idx], key);
            }
        }
    }
    // Scenario B: The key is NOT found in the current node. We need to descend to a child.
    else {
        // If we've reached a leaf node and the key was not found, it means the key is not in the tree.
        if (node->leaf) {
            // printf("Key %d not found in B-tree.\n", key); // Commented to reduce excessive output for 1M ops
            return node; // No changes needed if key not found in leaf
        }

        // Determine if the child to descend into is the last child. This is important
        // because merging operations might change the indices of children.
        int is_last_child = (idx == node->num_keys) ? 1 : 0;

        BTreeNode *child = node->children[idx]; // The child pointer to follow.

        // Pre-check: If the child we are about to descend into is deficient (fewer than ORDER keys),
        // we must "fill" it first by borrowing from a sibling or merging with one.
        // This ensures that when we return from the recursive call, the child won't be
        // even more deficient (e.g., empty) and cause issues up the tree.
        if (child->num_keys < ORDER) {
            btree_fill(node, idx);
        }

        // After `btree_fill` (which might have merged `child` with a sibling),
        // re-evaluate which child to descend into. If `is_last_child` was true
        // and a merge occurred, the correct child might now be `idx - 1`.
        if (is_last_child && node->num_keys > idx) {
            // The original `idx` was the last child, and a merge occurred
            // (likely with the left sibling, making `idx-1` the new target).
            node->children[idx - 1] = btree_remove_recursive(node->children[idx - 1], key);
        } else {
            // Otherwise, descend into the child at `idx`.
            node->children[idx] = btree_remove_recursive(node->children[idx], key);
        }
    }

    return node; // Return the (potentially modified) current node.
}

/* ===================== B-Tree Public Remove Function ===================== */

/**
 * @brief Public interface to remove a key from the B-Tree.
 * This function serves as the entry point for deletion and handles the
 * special case where the root of the tree might become empty after a deletion,
 * in which case its only child becomes the new root.
 * @param root A pointer to the pointer to the root node of the B-Tree.
 * This allows the root pointer in `main` to be updated.
 * @param key The key to be removed.
 */
void btree_remove(BTreeNode **root, int key) {
    if (!(*root)) {
        //printf("Tree is empty. Cannot remove %d.\n", key); // Commented to reduce excessive output
        return;
    }

    // Call the main recursive removal function.
    // The recursive function returns the (potentially modified) root node.
    *root = btree_remove_recursive(*root, key);

    // After deletion, check if the root has become empty (0 keys) and is not a leaf.
    // This typically happens when the root had only one child, and a merge operation
    // caused that child to become the new root.
    if ((*root) && (*root)->num_keys == 0 && !(*root)->leaf) {
        BTreeNode *temp = *root;           // Store the current empty root.
        *root = (*root)->children[0];      // Its single child becomes the new root.
        free(temp);                        // Free the old empty root node to prevent memory leaks.
    }
}

/* ===================== Memory Cleanup ===================== */

/**
 * @brief Recursively frees all nodes in the B-tree.
 * This is crucial to prevent memory leaks, especially for large trees.
 * @param node Pointer to the current node to free.
 */
void btree_destroy(BTreeNode *node) {
    if (node == NULL) {
        return;
    }
    if (!node->leaf) {
        for (int i = 0; i <= node->num_keys; ++i) {
            btree_destroy(node->children[i]);
        }
    }
    free(node);
}

/* ===================== Dynamic List Helper Functions ===================== */
// These functions manage the dynamic arrays used to store operations parsed from the input file.

/**
 * @brief Dynamically resizes a generic integer array.
 * @param arr Pointer to the array pointer.
 * @param count Current number of elements.
 * @param capacity Current capacity of the array.
 * @param min_needed_capacity The minimum required capacity.
 */
void resize_int_array(int **arr, long long *count, long long *capacity, long long min_needed_capacity) {
    if (*count >= *capacity) {
        long long new_cap = (*capacity == 0) ? NUM_OPERATIONS : *capacity * 2; // Initial allocation or double current
        if (new_cap < min_needed_capacity) { // Ensure enough capacity for at least one more element
             new_cap = min_needed_capacity;
        }
        int *new_arr = realloc(*arr, (size_t)new_cap * sizeof(int));
        if (!new_arr) {
            perror("Failed to reallocate integer array");
            exit(EXIT_FAILURE);
        }
        *arr = new_arr;
        *capacity = new_cap;
    }
}

/**
 * @brief Dynamically resizes an InsertOp array.
 * @param arr Pointer to the array pointer.
 * @param count Current number of elements.
 * @param capacity Current capacity of the array.
 * @param min_needed_capacity The minimum required capacity.
 */
void resize_insert_op_array(InsertOp **arr, long long *count, long long *capacity, long long min_needed_capacity) {
    if (*count >= *capacity) {
        long long new_cap = (*capacity == 0) ? NUM_OPERATIONS : *capacity * 2; // Initial allocation or double current
        if (new_cap < min_needed_capacity) { // Ensure enough capacity for at least one more element
             new_cap = min_needed_capacity;
        }
        InsertOp *new_arr = realloc(*arr, (size_t)new_cap * sizeof(InsertOp));
        if (!new_arr) {
            perror("Failed to reallocate InsertOp array");
            exit(EXIT_FAILURE);
        }
        *arr = new_arr;
        *capacity = new_cap;
    }
}

/**
 * @brief Adds an insert operation to the global list.
 * @param key The key to insert.
 * @param value The value to associate with the key.
 */
void add_insert_op(int key, int value) {
    resize_insert_op_array(&insert_ops_list, &insert_ops_count, &insert_ops_capacity, insert_ops_count + 1);
    insert_ops_list[insert_ops_count].key = key;
    insert_ops_list[insert_ops_count].value = value;
    insert_ops_count++;
}

/**
 * @brief Adds a key for a lookup operation to the global list.
 * @param key The key to lookup.
 */
void add_lookup_key(int key) {
    resize_int_array(&lookup_keys_list, &lookup_keys_count, &lookup_keys_capacity, lookup_keys_count + 1);
    lookup_keys_list[lookup_keys_count++] = key;
}

/**
 * @brief Adds a key for a lowerbound operation to the global list.
 * @param key The key for lowerbound search.
 */
void add_lowerbound_key(int key) {
    resize_int_array(&lowerbound_keys_list, &lowerbound_keys_count, &lowerbound_keys_capacity, lowerbound_keys_count + 1);
    lowerbound_keys_list[lowerbound_keys_count++] = key;
}

/**
 * @brief Adds a key for a remove operation to the global list.
 * @param key The key to remove.
 */
void add_remove_key(int key) {
    resize_int_array(&remove_keys_list, &remove_keys_count, &remove_keys_capacity, remove_keys_count + 1);
    remove_keys_list[remove_keys_count++] = key;
}

/* ===================== File Parsing and Collection ===================== */

/**
 * @brief Reads a single file, parses commands, and collects them into global lists.
 * This function does NOT perform any actual index operations; it only populates
 * the operation lists.
 * @param filename The path to the input file containing benchmark commands.
 */
void parse_and_collect_operations(const char *filename) {
    FILE *fp = fopen(filename, "r");
    if (!fp) {
        fprintf(stderr, "ERROR: Failed to open file %s. Please ensure it exists and is accessible.\n", filename);
        exit(EXIT_FAILURE); // Exit if the main input file cannot be opened
    }

    char line[LINE_LEN];
    long long line_num = 0;
    printf("Parsing and collecting operations from file: %s...\n", filename);

    while (fgets(line, sizeof(line), fp)) {
        line_num++;
        // Remove trailing newline or carriage return
        line[strcspn(line, "\r\n")] = 0;

        if (strncmp(line, "insert", 6) == 0) {
            int k, v;
            if (sscanf(line, "insert %d %d", &k, &v) == 2) {
                add_insert_op(k, v);
                // Consume the 'ok' line that follows 'insert' in benchmark files
                if (!fgets(line, sizeof(line), fp)) {
                    fprintf(stderr, "ERROR: Unexpected EOF after insert command on line %lld in %s\n", line_num, filename);
                    break;
                }
                line_num++; // Increment line_num for the 'ok' line
            } else {
                fprintf(stderr, "ERROR: Malformed insert line (expected 'insert <key> <value>') on line %lld in %s: '%s'\n", line_num, filename, line);
            }
        }
        else if (strncmp(line, "lookup", 6) == 0) {
            int k;
            if (sscanf(line, "lookup %d", &k) == 1) {
                add_lookup_key(k);
                // Consume the output line (e.g., "840290302" or "null")
                if (!fgets(line, sizeof(line), fp)) {
                    fprintf(stderr, "ERROR: Unexpected EOF after lookup command on line %lld in %s\n", line_num, filename);
                    break;
                }
                line_num++; // Increment line_num for the output line
            } else {
                fprintf(stderr, "ERROR: Malformed lookup line (expected 'lookup <key>') on line %lld in %s: '%s'\n", line_num, filename, line);
            }
        } else if (strncmp(line, "remove", 6) == 0) {
            int k;
            if (sscanf(line, "remove %d", &k) == 1) {
                add_remove_key(k);
                // Consume the 'ok' line that follows 'remove'
                if (!fgets(line, sizeof(line), fp)) {
                    fprintf(stderr, "ERROR: Unexpected EOF after remove command on line %lld in %s\n", line_num, filename);
                    break;
                }
                line_num++; // Increment line_num for the 'ok' line
            } else {
                fprintf(stderr, "ERROR: Malformed remove line (expected 'remove <key>') on line %lld in %s: '%s'\n", line_num, filename, line);
            }
        } else if (strncmp(line, "lowerbound", 10) == 0) {
            int k;
            if (sscanf(line, "lowerbound %d", &k) == 1) {
                add_lowerbound_key(k);
                // Consume the output line
                if (!fgets(line, sizeof(line), fp)) {
                    fprintf(stderr, "ERROR: Unexpected EOF after lowerbound command on line %lld in %s\n", line_num, filename);
                    break;
                }
                line_num++; // Increment line_num for the output line
            } else {
                fprintf(stderr, "ERROR: Malformed lowerbound line (expected 'lowerbound <key>') on line %lld in %s: '%s'\n", line_num, filename, line);
            }
        } else if (strlen(line) > 0 && line[0] != '\n' && line[0] != '\r') { // Ignore truly empty or newline-only lines
            // fprintf(stderr, "WARNING: Unrecognized or invalid command on line %lld in %s: '%s'\n", line_num, filename, line); // Commented to reduce excessive output
        }
    }
    fclose(fp);
    printf("Finished parsing. Collected %lld inserts, %lld lookups, %lld lowerbounds, %lld removals.\n",
           insert_ops_count, lookup_keys_count, lowerbound_keys_count, remove_keys_count);
}

/* ===================== Benchmark Execution Helper Functions ===================== */

// Helper function to calculate time difference in seconds using timespec (for POSIX)
double get_elapsed_time(struct timespec start, struct timespec end) {
    return (double)(end.tv_sec - start.tv_sec) + (double)(end.tv_nsec - start.tv_nsec) / 1e9;
}

/**
 * @brief Runs the insert benchmark phase using collected operations.
 * @param alex Pointer to the ALEXLeaf structure.
 * @param btree Pointer to the root of the B+Tree.
 */
void run_insert_benchmark(ALEXLeaf *alex, BTreeNode **btree) {
    struct timespec start, end;
    double alex_elapsed, btree_elapsed;

    printf("\n--- Starting Insert Benchmark (%lld operations) ---\n", insert_ops_count);

    // Measure ALEX inserts
    printf("  Measuring ALEX inserts...\n");
    clock_gettime(CLOCK_MONOTONIC, &start);
    for (long long i = 0; i < insert_ops_count; ++i) {
        alex_insert(alex, insert_ops_list[i].key, insert_ops_list[i].value);
    }
    clock_gettime(CLOCK_MONOTONIC, &end);
    alex_elapsed = get_elapsed_time(start, end);
    printf("  ALEX Insert time: %.6f seconds\n", alex_elapsed);

    // Measure B-tree inserts
    printf("  Measuring B-tree inserts...\n");
    clock_gettime(CLOCK_MONOTONIC, &start);
    for (long long i = 0; i < insert_ops_count; ++i) {
        btree_insert(btree, insert_ops_list[i].key, insert_ops_list[i].value);
    }
    clock_gettime(CLOCK_MONOTONIC, &end);
    btree_elapsed = get_elapsed_time(start, end);
    printf("  B-tree Insert time: %.6f seconds\n", btree_elapsed);

    printf("--- Insert Benchmark Completed ---\n");
}

/**
 * @brief Runs the lookup benchmark phase using collected operations.
 * @param alex Pointer to the ALEXLeaf structure.
 * @param btree Pointer to the root of the B+Tree.
 */
void run_lookup_benchmark(ALEXLeaf *alex, BTreeNode **btree) {
    struct timespec start, end;
    double alex_elapsed, btree_elapsed;

    printf("\n--- Starting Lookup Benchmark (%lld operations) ---\n", lookup_keys_count);

    // Measure ALEX lookups
    printf("  Measuring ALEX lookups...\n");
    clock_gettime(CLOCK_MONOTONIC, &start);
    for (long long i = 0; i < lookup_keys_count; ++i) {
        int key_to_lookup = lookup_keys_list[i];
        alex_lookup(alex, key_to_lookup);
    }
    clock_gettime(CLOCK_MONOTONIC, &end);
    alex_elapsed = get_elapsed_time(start, end);
    printf("  ALEX Lookup time: %.6f seconds\n", alex_elapsed);

    // Measure B-tree lookups
    printf("  Measuring B-tree lookups...\n");
    clock_gettime(CLOCK_MONOTONIC, &start);
    for (long long i = 0; i < lookup_keys_count; ++i) {
        int key_to_lookup = lookup_keys_list[i];
        btree_lookup(*btree, key_to_lookup);
    }
    clock_gettime(CLOCK_MONOTONIC, &end);
    btree_elapsed = get_elapsed_time(start, end);
    printf("  B-tree Lookup time: %.6f seconds\n", btree_elapsed);

    printf("--- Lookup Benchmark Completed ---\n");
}

/**
 * @brief Runs the lowerbound benchmark phase using collected operations.
 * @param alex Pointer to the ALEXLeaf structure.
 * @param btree Pointer to the root of the B+Tree.
 */
void run_lowerbound_benchmark(ALEXLeaf *alex, BTreeNode **btree) {
    struct timespec start, end;
    double alex_elapsed, btree_elapsed;

    printf("\n--- Starting Lowerbound Benchmark (%lld operations) ---\n", lowerbound_keys_count);

    // Measure ALEX lowerbounds
    printf("  Measuring ALEX lowerbounds...\n");
    clock_gettime(CLOCK_MONOTONIC, &start);
    for (long long i = 0; i < lowerbound_keys_count; ++i) {
        int key_to_lowerbound = lowerbound_keys_list[i];
        alex_lowerbound(alex, key_to_lowerbound);
    }
    clock_gettime(CLOCK_MONOTONIC, &end);
    alex_elapsed = get_elapsed_time(start, end);
    printf("  ALEX Lowerbound time: %.6f seconds\n", alex_elapsed);

    // Measure B-tree lowerbounds
    printf("  Measuring B-tree lowerbounds...\n");
    clock_gettime(CLOCK_MONOTONIC, &start);
    for (long long i = 0; i < lowerbound_keys_count; ++i) {
        int key_to_lowerbound = lowerbound_keys_list[i];
        btree_lowerbound(*btree, key_to_lowerbound);
    }
    clock_gettime(CLOCK_MONOTONIC, &end);
    btree_elapsed = get_elapsed_time(start, end);
    printf("  B-tree Lowerbound time: %.6f seconds\n", btree_elapsed);

    printf("--- Lowerbound Benchmark Completed ---\n");
}

/**
 * @brief Runs the remove benchmark phase using collected operations.
 * @param alex Pointer to the ALEXLeaf structure.
 * @param btree Pointer to the root of the B+Tree.
 */
void run_remove_benchmark(ALEXLeaf *alex, BTreeNode **btree) {
    struct timespec start, end;
    double alex_elapsed, btree_elapsed;

    printf("\n--- Starting Remove Benchmark (%lld operations) ---\n", remove_keys_count);

    // Measure ALEX removes
    printf("  Measuring ALEX removals...\n");
    clock_gettime(CLOCK_MONOTONIC, &start);
    for (long long i = 0; i < remove_keys_count; ++i) {
        int key_to_remove = remove_keys_list[i];
        alex_remove(alex, key_to_remove);
    }
    clock_gettime(CLOCK_MONOTONIC, &end);
    alex_elapsed = get_elapsed_time(start, end);
    printf("  ALEX Remove time: %.6f seconds\n", alex_elapsed);

    // Measure B-tree removes
    printf("  Measuring B-tree removals...\n");
    clock_gettime(CLOCK_MONOTONIC, &start);
    for (long long i = 0; i < remove_keys_count; ++i) {
        int key_to_remove = remove_keys_list[i];
        btree_remove(btree, key_to_remove);
    }
    clock_gettime(CLOCK_MONOTONIC, &end);
    btree_elapsed = get_elapsed_time(start, end);
    printf("  B-tree Remove time: %.6f seconds\n", btree_elapsed);

    printf("--- Remove Benchmark Completed ---\n");
}

/* ===================== Main Function ===================== */

int main(int argc, char *argv[]) {
    // Seed the random number generator for any potential random operations (e.g., in future random workloads)
    srand((unsigned int)time(NULL));

    // Check for correct number of command-line arguments (program name + 1 input file)
    if (argc != 2) {
        printf("Usage: %s <benchmark_operations_file>\n", argv[0]);
        printf("The <benchmark_operations_file> should contain all insert, lookup, lowerbound, and remove commands, with 'ok' or output lines as specified in the comments.\n");
        return 1;
    }

    printf("--- Initializing Data Structures ---\n");

    // Initialize ALEX structure (dynamically allocated, capacity set by LEAF_CAPACITY)
    ALEXLeaf *alex = (ALEXLeaf *)malloc(sizeof(ALEXLeaf));
    if (!alex) {
        perror("Failed to allocate ALEXLeaf structure in main");
        return 1;
    }
    alex_init(alex, LEAF_CAPACITY); // Initialize ALEXLeaf with the desired capacity (1,000,000)

    // Initialize B-tree root as a leaf node
    BTreeNode *btree = create_btree_node(1);

    // Parse and collect all operations from the input file before running any benchmarks.
    // This ensures both data structures operate on the exact same sequence of operations.
    printf("\n--- Parsing Input File ---\n");
    parse_and_collect_operations(argv[1]);
    printf("--- Input File Parsing Completed ---\n");

    printf("\n--- Starting Combined Benchmarks ---\n");
    struct timespec total_start, total_end;
    clock_gettime(CLOCK_MONOTONIC, &total_start);

    // Run each benchmark type sequentially for both data structures
    printf("\n>>> Running Insert Benchmark Phase <<<\n");
    run_insert_benchmark(alex, &btree);
    printf("Insert Benchmark Phase Completed.\n");

    printf("\n>>> Running Lookup Benchmark Phase <<<\n");
    run_lookup_benchmark(alex, &btree);
    printf("Lookup Benchmark Phase Completed.\n");

    printf("\n>>> Running Lowerbound Benchmark Phase <<<\n");
    run_lowerbound_benchmark(alex, &btree);
    printf("Lowerbound Benchmark Phase Completed.\n");

    printf("\n>>> Running Remove Benchmark Phase <<<\n");
    run_remove_benchmark(alex, &btree);
    printf("Remove Benchmark Phase Completed.\n");

    clock_gettime(CLOCK_MONOTONIC, &total_end);
    printf("\n--- All Combined Benchmarks Completed. Total runtime: %.6f seconds ---\n", get_elapsed_time(total_start, total_end));


    printf("\n--- Starting Memory Cleanup ---\n");

    // Free all collected operation lists to prevent memory leaks
    printf("Freeing collected operation lists...\n");
    if (insert_ops_list) { free(insert_ops_list); insert_ops_list = NULL; }
    if (lookup_keys_list) { free(lookup_keys_list); lookup_keys_list = NULL; }
    if (lowerbound_keys_list) { free(lowerbound_keys_list); lowerbound_keys_list = NULL; }
    if (remove_keys_list) { free(remove_keys_list); remove_keys_list = NULL; }
    printf("Collected operation lists freed.\n");

    // Comprehensive cleanup: Free all B+Tree nodes recursively
    if (btree) {
        printf("Destroying B-tree...\n");
        btree_destroy(btree);
        btree = NULL;
        printf("B-tree destroyed.\n");
    }

    // Clean up ALEX data structure
    if (alex) {
        printf("Destroying ALEXLeaf structure...\n");
        alex_destroy(alex); // This frees internal arrays of the leaf
        free(alex); // Free the ALEXLeaf structure itself
        alex = NULL;
        printf("ALEXLeaf structure destroyed.\n");
    }

    printf("\nAll memory cleanup completed. Program exit.\n");

    return 0;
}


Writing alex_btree_combined_benchmark.c


In [2]:
!gcc alex_btree_combined_benchmark.c -o alex_btree

In [ ]:
!./alex_btree combined_benchmark_2m_ops.txt

--- Initializing Data Structures ---

--- Parsing Input File ---
Parsing and collecting operations from file: combined_benchmark_2m_ops.txt...
Finished parsing. Collected 1000000 inserts, 1000000 lookups, 1000000 lowerbounds, 1000000 removals.
--- Input File Parsing Completed ---

--- Starting Combined Benchmarks ---

>>> Running Insert Benchmark Phase <<<

--- Starting Insert Benchmark (1000000 operations) ---
  Measuring ALEX inserts...


In [3]:
!./alex_btree combined_benchmark_200k_ops.txt

--- Initializing Data Structures ---

--- Parsing Input File ---
Parsing and collecting operations from file: combined_benchmark_200k_ops.txt...
Finished parsing. Collected 100000 inserts, 100000 lookups, 100000 lowerbounds, 100000 removals.
--- Input File Parsing Completed ---

--- Starting Combined Benchmarks ---

>>> Running Insert Benchmark Phase <<<

--- Starting Insert Benchmark (100000 operations) ---
  Measuring ALEX inserts...
  ALEX Insert time: 11.031961 seconds
  Measuring B-tree inserts...
  B-tree Insert time: 0.027506 seconds
--- Insert Benchmark Completed ---
Insert Benchmark Phase Completed.

>>> Running Lookup Benchmark Phase <<<

--- Starting Lookup Benchmark (100000 operations) ---
  Measuring ALEX lookups...
  ALEX Lookup time: 20.023125 seconds
  Measuring B-tree lookups...
  B-tree Lookup time: 0.024724 seconds
--- Lookup Benchmark Completed ---
Lookup Benchmark Phase Completed.

>>> Running Lowerbound Benchmark Phase <<<

--- Starting Lowerbound Benchmark (10000